In [2]:
import re
from greek_normalisation.utils import (grave_to_acute,strip_last_accent_if_two)

lexicalentries = {}
verbarguments = {}
lexfile = "/Users/gcrane/github/Thucydides-new-working-materials/GreekMorphApril18.csv"
with open(lexfile) as f:
    for line in f:

        #print(line)
        #line = line.strip('"')
        fields = line.split(",")
        fields[1] = fields[1].strip('"')
        fields[2] = fields[2].strip('\n')
        fields[2] = fields[2].strip('"')
        lexicalentries[fields[1]] = fields[2]

f.close()

anabasistb = '/Users/gcrane/github/glaux-trees/public/xml/0032-006.xml'
republictb = '/Users/gcrane/github/glaux-trees/public/xml/0059-030.xml'
iliadtb = '/Users/gcrane/github/gAGDT/data/xml/tlg0012.tlg001.perseus-grc1.tb.xml'

curtb = anabasistb
#curtb = iliadtb

greeksentids = {}
greeksentforms = {}
greeksentlems = {}
greeksentlens = {}
greeksentpostags = {}
greeksentheads = {}
greeksentrels = {}
engsents = {}
engsentcits = {}
engsentlens = {}

greekforms = {}
greeklems = {}
hashead = {}
hasdeps = {}


neutvocsg = 's---nv'
neutnomsg = 's---nn'
neutgensg = 's---ng'
neutdatsg = 's---nd'
neutaccsg = 's---na'

neutvocpl = 'p---nv'
neutnompl = 'p---nn'
neutgenpl = 'p---ng'
neutdatpl = 'p---nd'
neutaccpl = 'p---na'

femvocsg = 's---fv'
femnomsg = 's---fn'
femgensg = 's---fg'
femdatsg = 's---fd'
femaccsg = 's---fa'

femvocpl = 'p---fv'
femnompl = 'p---fn'
femgenpl = 'p---fg'
femdatpl = 'p---fd'
femaccpl = 'p---fa'

mascvocsg = 's---mv'
mascnomsg = 's---mn'
mascgensg = 's---mg'
mascdatsg = 's---md'
mascaccsg = 's---ma'

mascvocpl = 'p---mv'
mascnompl = 'p---mn'
mascgenpl = 'p---mg'
mascdatpl = 'p---md'
mascaccpl = 'p---ma'

commvocsg = 's---cv'
commnomsg = 's---cn'
commgensg = 's---cg'
commdatsg = 's---cd'
commaccsg = 's---ca'

commvocpl = 'p---cv'
commnompl = 'p---cn'
commgenpl = 'p---cg'
commdatpl = 'p---cd'
commaccpl = 'p---ca'


mascparad = [mascnomsg,mascgensg,mascdatsg,mascaccsg,mascnompl,mascgenpl,mascdatpl,mascaccpl]

defart_forms = {
                mascvocsg : 'ὦ', mascnomsg : 'ὁ',  mascgensg : 'τοῦ', mascdatsg :'τῷ',  mascaccsg :'τὸν', 
                mascvocpl : 'ὦ', mascnompl: 'οἱ', mascgenpl : 'τῶν',mascdatpl : 'τοῖς', mascaccpl : 'τοὺς',
                femvocsg : 'ὦ', femnomsg : 'ἡ',  femgensg : 'τῆς', femdatsg :'τῇ',  femaccsg :'τὴν', 
                femvocpl : 'ὦ', femnompl: 'αἱ', femgenpl : 'τῶν',femdatpl : 'ταῖς', femaccpl : 'τὰς',
                neutvocsg : 'ὦ', neutnomsg : 'τὸ',  neutgensg : 'τοῦ', neutdatsg :'τῷ',  neutaccsg :'τὸ', 
                neutvocpl : 'ὦ', neutnompl: 'τὰ', neutgenpl : 'τῶν',neutdatpl : 'τοῖς', neutaccpl : 'τὰ',
                commvocsg : 'ὦ', commnomsg : 'ὁ/ἡ/τὸ',  commgensg : 'τοῦ/τῆς', commdatsg :'τῷ/τῇ',  commaccsg :'τὸν/τὴν/τὸ', 
                commvocpl : 'ὦ', commnompl: 'οἱ/αἱ/τὰ', commgenpl : 'τῶν',commdatpl : 'τοῖς/ταῖς', commaccpl : 'τὰ/τὰς/τοὺς'
               }

lemma_pos_forms = {}



def getengsents(fname):
    f = open(fname)
    for l in f:
        l = re.sub('\n','',l)
        l = re.sub('"','',l)
        args = l.split('\t')
        if(not len(args) == 3):
            print(len(args),l)
            continue
        engsents[args[1]] = args[2]
        engsentcits[args[1]] = args[0]
    
    f.close()
    
def additem(curdict,curitem):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1
        

getengsents('/Users/gcrane/github/anabasis/sentfile.txt')  

depsdict = {}
formdict = {}
citdict = {}
sentdict = {}
def additem2list(curdict,hasdeps,newdep):
    if(not hasdeps in curdict):
        curdict[hasdeps] = []
    curdict[hasdeps].append(newdep)

def getfeats(curid):
    curform = ''
    if(curid in formdict):
        curform = formdict[curid]
    #print('gf',curid,curform)
    
    curpos = ''
    m = re.search('postag="([^"]+)"',curform)
    if(m):
        curpos = m[1]

    curlemma = ''
    m = re.search('lemma="([^"]+)"',curform)
    if(m):
        curlemma = m[1]

    curhead = ''
    m = re.search('head="([^"]+)"',curform)
    if(m):
        curhead = m[1]

    curinfl = ''
    m = re.search('form="([^"]+)"',curform)
    if(m):
        curinfl = m[1]

    currel = ''
    m = re.search('relation="([^"]+)"',curform)
    if(m):
        currel = m[1]
    return(curpos,curlemma,curinfl,curhead,currel)


def getgreeksents(fname):
    f = open(fname)
    cursent = ''
    i = 0
    for l in f:
        l = re.sub('\n','',l)
        m = re.search('<sentence id="([0-9]+)"',l)
        if(m):
            if(cursent):
                i = i + 1
                #if(i<100):
                #    print(l)
                #    print(' '.join(greeksentforms[cursent]))
            cursent = m[1]
            greeksentforms[cursent] = []
            greeksentlems[cursent] = []
            greeksentpostags[cursent] = []
            greeksentrels[cursent] = []
        m = re.search('subdoc="([^"]+)"',l)
        if(m):
            curcit = m[1]
        m = re.search('word id="([^"]+)',l)
        if(m):
            
            curid = m[1]
            formdict[curid] = l
            curpostag,curlemma,curform,curhead,currel = getfeats(curid)
            #curform = m[2]
            #curlemma = m[3]
            #curpostag = m[4]
            #curhead = m[5]
            #currel = m[6]
            #print('a',curid,curlemma)
            hashead[curid] = curhead
            if(curhead in hasdeps):
                hasdeps[curhead].append(curid)
            else:
                hasdeps[curhead] = [curid]
            
            if(not curlemma in lemma_pos_forms):
                lemma_pos_forms[curlemma] = {}
            additem(lemma_pos_forms[curlemma], curpostag + ':' + strip_last_accent_if_two(grave_to_acute(curform)))
            
            greeksentforms[cursent].append(curform)
            additem(greekforms,curform)
            greeksentlems[cursent].append(curlemma)
            additem(greeklems,curlemma)
            greeksentpostags[cursent].append(curpostag)
            greeksentrels[cursent].append(currel)
            if(not curhead == '0'):
                additem2list(depsdict,curhead,curid)
            citdict[curid] = curcit
    f.close()
    print('len',len(lemma_pos_forms))
    for foo in greeksentlems:
        greeksentlens[foo] = len(greeksentlems[foo])

def get_relation(l):
    m = re.search('relation="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')
    
def get_postag(l):
    m = re.search('postag="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')
    
def get_lemma(l):
    m = re.search('lemma="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')

def get_curid(l):
    m = re.search('id="([^"]+)"',l)
    if(m):
        return(m[1])
    else:
        return('')

def is_decl1(curlemma):
    if(re.search('(ος|ός)$',curlemma)):
        return(1)
    return(0)

def has_matchingarticle(headid,matchpos):
    if( not headid in depsdict):
        return
    for foo in depsdict[headid]:
        l = formdict[foo]
        if(re.search('lemma="ὁ"',l) and re.search('relation="ATR',l) and re.search(matchpos,l)):
            return(get_curid(l))
    return('')
 
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

fname = curtb

getgreeksents(curtb)
sword = 'στρατηγός'
sword = 'ἔχει'
sword = 'Δαρεικός'
sword = 'ἦν'
sword = 'ἦσαν'
sword = 'ἔχει'
sword = 'πέμπουσι'
sword = 'στρατηγός'
sword = 'ἀδελφός'
sword = 'ποταμός'
sword = 'κώμη'


len 3694


In [14]:
optatives = []
perfects = []
aoristpass = []
comparatives = []
superlatives = []
mediopassives = []
miverbs = []
genabsolutes = []
allverbs = {}
totforms = 0

preplist = {}
prepusages = []
prep_acc = {}
prep_gen = {}
prep_dat = {}

outfname = 'tbankout.txt'
def prepdep(curid,curhead):
    #print(curid,curhead)
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    if(headlemma == '(καί|,)' or re.search('^[bu]',headpos)):
        #print('kai')
        curhead = headhead
        headpos,headlemma,headform,headhead,headrel = getfeats(curhead)

    if(curlemma == '(καί|,)' or re.search('^[bu]',curpos)):
        return

    if(re.search('^r',headpos)):
        m = re.search('([a-z].)$',curpos)
        if(m):
            curcase = m[1]
        else:
            curcase = 'x'
        #print(headlemma,curcase,curform,curlemma,curpos,citdict[curid])
        additem(preplist,headlemma)
        prepusages.append([headlemma,curcase,curform,curlemma,curpos,citdict[curid]])
        if(curcase[0] == 'a'):
            additem(prep_acc,headlemma)
        elif(curcase[0] == 'g'):
            additem(prep_gen,headlemma)
        elif(curcase[0] == 'd'):
            additem(prep_dat,headlemma)
noungennouns = []

def getnoungennouns(curid):
    global noungennouns
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('^n.+g-$',curpos) and currel == 'ATR' and re.search('^[pn]',headpos)):
        noungennouns.append([len(noungennouns)+1,curform,currel,headform,headlemma,curid,citdict[curid]])
        #print('ngn',len(noungennouns),curform,currel,headform,headlemma,curid,citdict[curid])

adj2dats = []

def getadj2dats(curid):
    global adj2dats
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('d-$',curpos) and currel == 'ADV' and re.search('^a',headpos)):
        adj2dats.append([len(adj2dats)+1,curform,currel,headform,headlemma,curid,citdict[curid]])
        #print('a2d',len(adj2dats),curform,currel,headform,headlemma,curid,citdict[curid])


def hasarticle(curid):
    for foo in hasdeps[curid]:
        if(foo == '0'):
            continue
        if(re.search('postag="l',formdict[foo])):
            return(foo)
    return('')
            
def autos(curid):
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    gnc1 = ''
    gnc2 = ''
    m = re.search('^..([sdp])...([mnfc][nvadg])',curpos)
    if(m):
        gnc1 = m[1] + m[2]
        
    m = re.search('^..([sdp])...([mnfc][nvadg])',headpos)
    if(m):
        gnc2 = m[1] + m[2]
    art = hasarticle(curhead)
    #if(curlemma == 'αὐτός' and gnc1 == gnc2):
     #   print('autos2','gnc1='+gnc1,'a'+art,curcit,gnc1,getfeats(curid),gnc2,getfeats(curhead))
    if(gnc1 and gnc1 == gnc2 and curlemma == 'αὐτός'):
        if(art):
            artn = int(art)
            autosn = int(curid)
            headn = int(curhead)
            # autos o Kuros
            #return
            if( artn > autosn and artn < headn ):
                print('\nintensive',curcit,formdict[art])
            # o autos  Kuros
            elif( artn < autosn and artn < headn ):
                print('\nthesame',curcit,formdict[art])
            else:
                print('\nnocat'+formdict[art])
        #else:
            
        #print('autos1',curid,'art='+art,int(curhead)-int(curid),curlemma,curform,headform,headlemma,curpos,headpos,curcit)
        
dmeans = 0
datmeansexamp = []

def datmeans(curid):
    global dmeans
    global datmeansexamp
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('d-$',curpos) and currel == 'ADV' and re.search('^v',headpos)):
        dmeans = dmeans + 1
        datmeansexamp.append([curform,currel,headform,curid,citdict[curid]])
        #print('dmeans',dmeans,curform,currel,headform,curid,citdict[curid])
        
iobjverbs = {}       
iobj = 0
def iobjs(curid):
    global iobj
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('d-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        iobj = iobj + 1
        if(not headlemma in iobjverbs):
            iobjverbs[headlemma] = {}
        curdict = iobjverbs[headlemma]
        additem(curdict,curlemma)
        #print('iobj',iobj,curform,currel,headform,headlemma,curid,citdict[curid])

conditionals = {}

def getconditional(curid):
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    headpos2,headlemma2,headform2,headhead2,headrel2 = getfeats(headhead)
    
    
    if(curpos[0] == 'v' and re.search('^(εἰ|ἐάν|ὅταν)$',headlemma)):
        anform2 = 'no-an'
        anlemma2 = 'no-an'
        if(headhead2 in hasdeps):
            for foo in hasdeps[headhead2]:
                deppos,deplemma,depform,dephead,deprel = getfeats(foo)
                if(deplemma == 'ἄν' or deplemma == 'ὅταν'):
                    anform2 = depform
                    anlemma2 = deplemma
        if(not anform2 == 'no-an' or 1):
            print('cond',curcit,curid,headform,curform,headform2,anform2)
    
    
gobjverbs = {}       
gobj = 0
def gobjs(curid):
    global gobj
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('g-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        gobj = gobj + 1
        if(not headlemma in gobjverbs):
            gobjverbs[headlemma] = {}
        curdict = gobjverbs[headlemma]
        additem(curdict,curlemma)
        #print('gobj',gobj,curform,currel,headform,headlemma,curid,citdict[curid])

verbarg = 0
def verbargs(curid,curverb,curcase,needrel):
    global verbarg
    subjform = 'nosbj'
    subjlemma = 'nosbj'
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(needrel and not currel == needrel):
        return
    if(re.search(curcase + '-$',curpos) and headlemma == curverb):
        verbarg = verbarg + 1
        for foo in hasdeps[curhead]:
            deppos,deplemma,depform,dephead,deprel = getfeats(foo)
            if(deprel == 'SBJ'):
                subjform = depform
                subjlemma = deplemma
        print('heads',verbarg,curform,currel,headform,headlemma,subjform,subjlemma,curid,citdict[curid])

tempepei = 0
tempcs = ['ἐπεί','ἐπειδή','πρίν']
def tempepeis(curid):
    global tempepei
    subjform = 'nosbj'
    subjlemma = 'nosbj'
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    headpos2,headlemma2,headform2,headhead2,headrel2 = getfeats(headhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    #if(not headrel == 'ADV' or not re.search('^v.+g-$',headpos) or not curpos[0] == 'v'):
    if(not headlemma in tempcs or not re.search('^v',curpos) or not re.search('^v',headpos2)):
        return
    
    else:
        tempepei = tempepei + 1
        if(headhead2 in hasdeps):
            for foo in hasdeps[headhead2]:
                deppos,deplemma,depform,dephead,deprel = getfeats(foo)
                if(deprel == 'SBJ'):
                    subjform = depform
                    subjlemma = deplemma
        print('tempepei',tempepei,headform,headlemma,curform,currel,headform2,headlemma2,headpos2,subjform,subjlemma,curid,citdict[curid])

genagent = 0
def genagents(curid):
    global genagent
    subjform = 'nosbj'
    subjlemma = 'nosbj'
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    headpos2,headlemma2,headform2,headhead2,headrel2 = getfeats(headhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    #if(not headrel == 'ADV' or not re.search('^v.+g-$',headpos) or not curpos[0] == 'v'):
    if(not headlemma == 'ὑπό' or not re.search('g-$',curpos) or not re.search('^v',headpos2)):
        return
    
    else:
        genagent = genagent + 1
        if(headhead2 in hasdeps):
            for foo in hasdeps[headhead2]:
                deppos,deplemma,depform,dephead,deprel = getfeats(foo)
                if(deprel == 'SBJ'):
                    subjform = depform
                    subjlemma = deplemma
        print('genagent',genagent,curform,currel,headform,headlemma,headform2,headlemma2,headpos2,subjform,subjlemma,curid,citdict[curid])



purpclause = 0
purpclausconjs = ['ἵνα','ὡς','ὅπως']
primarytenses = ['p','f','r','t']
secondarytenses = ['i','a','l']
def purpclauses(curid):
    global purpclause
    subjform = 'nosbj'
    subjlemma = 'nosbj'
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    headpos2,headlemma2,headform2,headhead2,headrel2 = getfeats(headhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    #if(not headrel == 'ADV' or not re.search('^v.+g-$',headpos) or not curpos[0] == 'v'):
    if(not headlemma in purpclausconjs or not re.search('^v...[so]',curpos) or not re.search('^v',headpos2)):
        return
    
    else:
        purpclause = purpclause + 1
        if(headhead2 in hasdeps):
            for foo in hasdeps[headhead2]:
                deppos,deplemma,depform,dephead,deprel = getfeats(foo)
                if(deprel == 'SBJ'):
                    subjform = depform
                    subjlemma = deplemma
        curtag = 'other' + headpos2[3] + curpos[4]
        if(headpos2[3] in primarytenses and curpos[4] == 's'):
            curtag = 'primary'
        elif(headpos2[3] in secondarytenses and curpos[4] == 'o'):
            curtag = 'secondary'
        elif(headpos2[3] in secondarytenses and curpos[4] == 's'):
            curtag = 'vivid'
            
        print('purpclause',purpclause,curtag,curform,currel,headform,headlemma,headform2,headlemma2,headpos2,subjform,subjlemma,curid,citdict[curid])


futpart = 0
def futparts(curid):
    global futpart
    subjform = 'nosbj'
    subjlemma = 'nosbj'
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    if( not re.search('v..fp',curpos)):
        return
    
    else:
        futpart = futpart + 1
        if(headhead in hasdeps):
            for foo in hasdeps[headhead]:
                deppos,deplemma,depform,dephead,deprel = getfeats(foo)
                if(deprel == 'SBJ'):
                    subjform = depform
                    subjlemma = deplemma
        print('futpart',futpart,curform,currel,headform,headlemma,subjform,subjlemma,curid,citdict[curid])

genab = 0
def genabs(curid):
    global genab
    subjform = 'nosbj'
    subjlemma = 'nosbj'
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    #if(not headrel == 'ADV' or not re.search('^v.+g-$',headpos) or not curpos[0] == 'v'):
    if(headrel == 'SBJ'):
        return
    if(not currel == 'ADV' or not re.search('^v.+g-$',curpos) or not re.search('^v',headpos)):
        return
    
    else:
        genab = genab + 1
        if(curid in hasdeps):
            for foo in hasdeps[curid]:
                deppos,deplemma,depform,dephead,deprel = getfeats(foo)
                if(deprel == 'SBJ'):
                    subjform = depform
                    subjlemma = deplemma
        print('genabs',genab,curform,currel,headform,headlemma,subjform,subjlemma,curid,citdict[curid])
        print('genabs',genab,curform,currel,headform,headlemma,subjform,subjlemma,curid,citdict[curid],file=outf)

        
aobjverbs = {} 
exw_objs = {}
aobjnoms = {}
aobj = 0
def aobjs(curid):
    global aobj
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('a-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        aobj = aobj + 1
        if(not headlemma in aobjverbs):
            aobjverbs[headlemma] = {}
        curdict = aobjverbs[headlemma]
        additem(curdict,curlemma)
        
        if(not curlemma in aobjnoms):
            aobjnoms[curlemma] = {}
        curdict = aobjnoms[curlemma]
        additem(curdict,headlemma)
        if(headlemma == 'ἔχω'):
            additem(exw_objs,curlemma)
            #aobjverbs[headlemma] = 1
            print('exw_objs',aobj,curform,currel,headform,headlemma,curid,citdict[curid])
            
        #print('aobj',aobj,curform,currel,headform,headlemma,curid,citdict[curid])

subj2verbs = {}        
def getsubj2verbs(curid):
    global subj2verbs
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    if(not currel == 'SBJ'):
        return
    if(len(headpos)<5):
        print('shortheadpos',headpos,curhead)
    lemkey = headlemma + '-' + headpos[5]
    if(not curlemma in subj2verbs):
        subj2verbs[curlemma] = {}
    additem(subj2verbs[curlemma],lemkey)
    
    
datobjs = {}
datverbs = {}
def finddatobjs(curid):
    global datobjs
    global datverbs
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    if(re.search('d-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        if(headlemma in datverbs):
            datverbs[headlemma] = datverbs[headlemma] + 1
        else:
            datverbs[headlemma] = 1
    
datpossess = []
tobeverbs = ['εἰμί','γίγνομαι']
def finddatpossess(curid):
    global datpossess
    global tobeverbs
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    if(re.search('d-$',curpos) and (currel == 'ADV' or currel =='OBJ') and headlemma in tobeverbs):
        print('datposs',citdict[curid],headform,headlemma,curform)
        datpossess.append([citdict[curid],headform,headlemma,curform,curid])
    
neutsubj = []
def findneutsubj(curid):
    global neutsubj
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    if(re.search('^..p---nn-$',curpos) and (currel =='SBJ') ):
        print('neutsubj2',citdict[curid],headform,headlemma,curform)
        neutsubj.append([citdict[curid],headform,headlemma,curform,curid])
    
doubleobjs = {}
hasobjs = {}
def finddoubleobjs(curid):
    global doubleobjs
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    
    #if(re.search('d-$',curpos) and re.search('ADV',currel) and re.search('^v',headpos)):
    if(re.search('a-$',curpos) and currel == 'OBJ' and re.search('^v',headpos)):
        curkey = curid + ' ' + curform + ' ' + curlemma
        if(curhead in hasobjs):
            hasobjs[curhead].append(curkey)
            #print('multobj',citdict[curid],len(hasobjs[curhead]),headform,headlemma,curhead,hasobjs[curhead])
        else:
            hasobjs[curhead] = [curkey]
            
            

outf = open(outfname,'w')
for foo in formdict:
    if(not re.search('^1\.',citdict[foo]) and 0):
        continue
    curpos,curlemma,curform,curhead,currel = getfeats(foo)
    if(curpos[0] == 'v'):
        if(curlemma in allverbs):
            allverbs[curlemma] = allverbs[curlemma] + 1
        else:
            allverbs[curlemma] = 1

    totforms = totforms + 1
    curf = formdict[foo]
        
    if(re.search('"[^"]+g-".+SBJ',curf)):
        genabsolutes.append(curf)
        print('genabs-short',len(genabsolutes),curlemma,curform,curid,citdict[foo])
    if(re.search('"v...o',curf)):
        optatives.append(curf)
    if(re.search('"v..[rl].',curf)):
        perfects.append(curf)
    if(re.search('"v..a.p',curf)):
        aoristpass.append(curf)
    elif(re.search('"v....[pme]',curf)):
        mediopassives.append(curf)
    if(re.search('c" head',curf)):
        comparatives.append(curf)
    if(re.search('lemma="[^"]+μι"',curf)):
        miverbs.append(curf)
    if(re.search('postag="[^"]+s"',curf)):
        superlatives.append(curf)
    m = re.search('head="([^"]+)"',curf)
    if(m):
        curhead = m[1]
    m = re.search('id="([^"]+)"',curf)
    if(m):
        curid = m[1]
    
    prepdep(curid,curhead)
    autos(curid)
    datmeans(curid)
    aobjs(curid)
    gobjs(curid)
    iobjs(curid)
    verbargs(curid,'περί','[d]','')
    getnoungennouns(curid)
    getadj2dats(curid)
    finddoubleobjs(curid)
    getsubj2verbs(curid)
    getconditional(curid)
    genabs(curid)
    genagents(curid)
    tempepeis(curid)
    futparts(curid)
    purpclauses(curid)
    finddatobjs(curid)
    finddatpossess(curid)
    findneutsubj(curid)
i = 0
outf.close()
def cntdict(curdict1,curkey):
    if(curkey in curdict1):
        curdict = curdict1[curkey]
    else:
        return(0,0)
    dictlen = len(curdict)
    dicttot = 0
    #print(dictlen,curdict)
    for boo in curdict:
        dicttot = dicttot + curdict[boo]
    return(dictlen,dicttot)

for foo in sorted(allverbs,key=allverbs.get,reverse=True):
    i = i + 1
    dats,dats2 = cntdict(iobjverbs,foo)
    accs,accs2 = cntdict(aobjverbs,foo)
    gens,gens2 = cntdict(gobjverbs,foo)

    
    print(i,allverbs[foo],foo,accs,dats,gens)
#for foo in sorted(preplist):
#    print(foo)
print(totforms)
print('mediopassives',len(optatives))
print('optative',len(optatives))
print('aoristpassive',len(aoristpass))
print('comparatives',len(comparatives))
print('superlatives',len(superlatives))
print('perfects',len(perfects))
print('miverbs',len(miverbs))

   
    

tempepei 1 ἐπεὶ ἐπεί ἠσθένει ADV ἐβούλετο βούλομαι v3siie--- nosbj nosbj 8996010 1.1.1
exw_objs 6 ὁπλίτας OBJ ἔχων ἔχω 8996067 1.1.2
tempepei 2 ἐπεὶ ἐπεί ἐτελεύτησε ADV διαβάλλει διαβάλλω v3spia--- nosbj nosbj 8996079 1.1.3
purpclause 1 otherpo ἐπιβουλεύοι OBJ ὡς ὡς διαβάλλει διαβάλλω v3spia--- nosbj nosbj 8996096 1.1.3
futpart 1 ἀποκτενῶν ADV συλλαμβάνει συλλαμβάνω ὁ ὁ 8996106 1.1.3
cond 1.1.4 8996139 ἢν δύνηται βασιλεύσει no-an
purpclause 2 secondary εἴησαν OBJ ὡς ὡς ἐπεμελεῖτο ἐπιμελέομαι v3siie--- nosbj nosbj 8996193 1.1.5
exw_objs 15 φυλακὰς OBJ εἶχε ἔχω 8996223 1.1.6
genabs 1 ἐπιβουλεύοντος ADV παρήγγειλε παραγγέλλω Τισσαφέρνους Τισσαφέρνης 8996240 1.1.6
genabs-short 1 Τισσαφέρνης Τισσαφέρνους 8996240 1.1.6

thesame 1.1.7     <word id="8996272" form="τὰ" lemma="ὁ" postag="l-p---na-" head="8996274" relation="ATR"/>
datposs 1.1.7 ἦν εἰμί αὐτῷ
genagent 1 ἀντιστασιωτῶν OBJ ὑπὸ ὑπό πιεζόμενος πιέζω v-sppemn- nosbj nosbj 8996489 1.1.10
tempepei 3 πρὶν πρίν συμβουλεύσηται ADV καταλῦσαι 

genabs-short 10 στενοχωρία στενοχωρίας 9000943 1.5.7
genabs 11 φανέντος ADV ἐπέστη ἐφίστημι στενοχωρίας στενοχωρία 9000947 1.5.7
tempepei 27 ἐπεὶ ἐπεί ἐδόκουν ADV ἐκέλευσε κελεύω v3saia--- nosbj nosbj 9000976 1.5.8
exw_objs 258 χιτῶνας OBJ ἔχοντες ἔχω 9001031 1.5.8
heads 1 τραχήλοις ATR περὶ περί nosbj nosbj 9001043 1.5.8
heads 2 χερσίν ATR περὶ περί nosbj nosbj 9001048 1.5.8
cond 1.5.9 9001151 εἴ ποιοῖτο ἦν no-an
exw_objs 265 ἃς OBJ εἶχον ἔχω 9001185 1.5.10
genabs 12 ἀμφιλεξάντων ADV ἐνέβαλεν ἐμβάλλω στρατιωτῶν στρατιώτης 9001235 1.5.11
genabs-short 11 στρατιώτης στρατιωτῶν 9001242 1.5.11

thesame 1.5.12     <word id="9001278" form="τῇ" lemma="ὁ" postag="l-s---fd-" head="9001281" relation="ATR"/>
genabs-short 12 κραυγή κραυγῆς 9001348 1.5.12
genabs 13 γενομένης ADV ἵησι ἵημι κραυγῆς κραυγή 9001350 1.5.12
genabs-short 13 αὐτός αὐτοῦ 9001467 1.5.14
genabs 14 δεήσαντος ADV λέγοι λέγω αὐτοῦ αὐτός 9001470 1.5.14
cond 1.5.16 9001542 εἰ συνάψετε κατακεκόψεσθαι no-an
genabs-short 14 ἡμέτερος 

tempepei 48 πρὶν πρίν καταλῦσαι ADV ἐφάνη φαίνω v3saip--- nosbj nosbj 9006470 1.10.19
futpart 16 σημανοῦντα ADV πέμπει πέμπω nosbj nosbj 9006551 2.1.2
exw_objs 508 ἃ OBJ εἶχον ἔχω 9006564 2.1.2
genabs 24 ὄντων ADV ἦλθε ἔρχομαι nosbj nosbj 9006581 2.1.3
cond 2.1.3 9006636 εἰ μέλλοιεν περιμένοιεν no-an
tempepei 49 ἐπεὶ ἐπεί τετελεύτηκεν ADV ἀπαγγέλλετε ἀπαγγέλλω v2ppia--- nosbj nosbj 9006675 2.1.4
cond 2.1.4 9006699 εἰ ἤλθετε ἐπορευόμεθα no-an
cond 2.1.4 9006712 ἐὰν ἔλθῃ καθιεῖν no-an
genabs 25 νικώντων ADV ἐστί εἰμί nosbj nosbj 9006725 2.1.4
tempepei 50 ἐπεὶ ἐπεί τυγχάνει ADV εὑρίσκεσθαι εὑρίσκω v--pne--- βασιλεὺς βασιλεύς 9006912 2.1.8
cond 2.1.8 9006927 ἄν δύνωνται εὑρίσκεσθαι no-an
genabs 26 νικώντων ADV εἴη εἰμί nosbj nosbj 9006953 2.1.9
exw_objs 529 ὅ OBJ ἔχετε ἔχω 9006972 2.1.9
purpclause 17 secondary ἴδοι ADV ὅπως ὅπως ἐκάλεσε καλέω v3saia--- nosbj nosbj 9006993 2.1.9
cond 2.1.10 9007052 εἰ κρατῶν δεῖ no-an
cond 2.1.10 9007066 εἰ βούλεται λεγέτω no-an
cond 2.1.10 9007078 ἐὰν χαρί

cond 2.4.17 9010681 ἐὰν δύνηται λῦσαι no-an
purpclause 25 primary διαβῆτε ADV ὡς ὡς διανοεῖται διανοέω v3spie--- nosbj nosbj 9010685 2.4.17
purpclause 26 secondary εἴη OBJ ὡς ὡς εἶπεν λέγω v3saia--- nosbj nosbj 9010727 2.4.19
cond 2.4.19 9010749 ἐὰν νικῶσι δεῖ no-an
cond 2.4.20 9010774 ἐὰν νικῶμεν ἕξουσιν no-an
genabs 38 λελυμένης ADV ἕξουσιν ἔχω γεφύρας γέφυρα 9010776 2.4.20
genabs-short 31 γέφυρα γεφύρας 9010777 2.4.20
genabs-short 32 πολύς πολλῶν 9010787 2.4.20
genabs 39 ὄντων ADV δυνήσεται δύναμαι πολλῶν πολύς 9010789 2.4.20
genabs 40 λελυμένης ADV δυνήσεται δύναμαι γεφύρας γέφυρα 9010794 2.4.20
genabs-short 33 γέφυρα γεφύρας 9010795 2.4.20
exw_objs 724 ἐρύματα OBJ ἔχοντες ἔχω 9010855 2.4.22
exw_objs 725 ἐπιτήδεια OBJ ἔχοιεν ἔχω 9010869 2.4.22
genabs 41 οὔσης ADV ἔχοιεν ἔχω nosbj nosbj 9010879 2.4.22
futpart 22 ἐργασομένων OBJ ἐνόντων ἔνειμι nosbj nosbj 9010882 2.4.22
cond 2.4.22 9010892 εἴ βούλοιτο γένοιτο no-an
purpclause 27 otherpo μέλλοιεν ADV ὡς ὡς διαβαινόντων διαβαίνω v-pppa

tempepei 90 ἐπεὶ ἐπεί ἔλυσαν ADV δοκεῖ δοκέω v3spia--- nosbj nosbj 9015131 3.1.21
neutsubj2 3.1.21 κεῖται κεῖμαι ἆθλα
exw_objs 937 σώματα OBJ ἔχομεν ἔχω 9015217 3.1.23
exw_objs 939 ψυχὰς OBJ ἔχομεν ἔχω 9015231 3.1.23
cond 3.1.23 9015255 ἢν διδῶσιν  no-an
cond 3.1.25 9015309 εἰ ἐθέλετε βούλομαι no-an
cond 3.1.25 9015321 εἰ τάττετ’ προφασίζομαι no-an
cond 3.1.26 9015378 εἰ δύναιτο πείσας no-an
tempepei 91 ἐπεὶ ἐπεί ἀπέθανε ADV ἐκέλευε κελεύω v3siia--- nosbj nosbj 9015419 3.1.27
tempepei 92 ἐπεὶ ἐπεί παρεσκηνήσαμεν ADV ἐποίησε ποιέω v3saia--- nosbj nosbj 9015440 3.1.28
tempepei 93 ἐπεὶ ἐπεί εἶδον ADV προσήκει προσήκω v3spia--- nosbj nosbj 9015585 3.1.31
tempepei 94 ἐπεὶ ἐπεί συνῆλθον ADV ἐκαθέζοντο καθέζομαι v3piie--- nosbj nosbj 9015640 3.1.33
purpclause 42 secondary βουλευσαίμεθα ADV ὅπως ὅπως παρακαλέσαι παρακαλέω v--ana--- nosbj nosbj 9015701 3.1.34
cond 3.1.34 9015704 εἴ δυναίμεθα βουλευσαίμεθα no-an
cond 3.1.35 9015756 ἢν δύνωνται ἀπολέσωσιν no-an
purpclause 43 primary ἀπολέσωσιν AD

...

genabs-short 135 σκότος σκότους 9054384 7.4.14
genabs 140 ὄντος ADV ἦν εἰμί σκότους σκότος 9054386 7.4.14
tempepei 329 ἐπεὶ ἐπεί ἐγένοντο ADV εἰσηκόντιζον εἰσακοντίζω v3piia--- nosbj nosbj 9054406 7.4.15
exw_objs 2885 ἃ OBJ ἔχειν ἔχω 9054424 7.4.15
futpart 94 ἀποκόψοντες ADV ἔφασαν φημί nosbj nosbj 9054428 7.4.15
exw_objs 2889 ἀσπίδας OBJ ἔχοντες ἔχω 9054469 7.4.16
genabs-short 136 αὐτός αὐτῶν 9054515 7.4.17
genabs 141 ὑπεραλλομένων ADV ἐλήφθησάν λαμβάνω αὐτῶν αὐτός 9054517 7.4.17
genabs 142 ἐνεχομένων ADV ἐλήφθησάν λαμβάνω πελτῶν πέλτη 9054523 7.4.17
genabs-short 137 πέλτη πελτῶν 9054524 7.4.17
exw_objs 2895 σαλπικτὴν OBJ ἔχων ἔχω 9054601 7.4.19
tempepei 330 ἐπεὶ ἐπεί ἦλθεν ADV ἐδεξιοῦτό δεξιόομαι v3siie--- nosbj nosbj 9054631 7.4.19
cond 7.4.20 9054659 εἰ βούλεται συστρατεύεσθαι no-an
exw_objs 2901 δύναμιν OBJ εἶχε ἔχω 9054704 7.4.21
futpart 95 συστρατευσόμενοι ADV κατέβαινον καταβαίνω nosbj nosbj 9054719 7.4.21
tempepei 331 ἐπεὶ ἐπεί εἶδον ADV ἱκέτευον ἱκετεύω v3piia--- nosbj nosbj 

futpart 99 βοηθήσοντες ADV ἐρχόμεθα ἔρχομαι nosbj nosbj 9058177 7.7.17
futpart 100 τιμωρησόμενοι CO βοηθήσοντες βοηθέω nosbj nosbj 9058180 7.7.17
cond 7.7.17 9058195 ἢν ἦτε ἀρξόμεθα no-an
tempepei 344 ἐπειδὴ ἐπειδή ἔφατε ADV ἐπιτρέψαι ἐπιτρέπω v--ana--- nosbj nosbj 9058219 7.7.18
cond 7.7.18 9058231 ἂν ψηφίσωνται ἐσμέν no-an
futpart 101 ἀπαιτήσων ADV πάρειμι πάρειμι nosbj nosbj 9058309 7.7.20
futpart 102 διδάξων ADV πάρειμι πάρειμι nosbj nosbj 9058317 7.7.20
cond 7.7.20 9058320 ἢν δύνωμαι διδάξων no-an
tempepei 345 ἐπεί ἐπεί ἐποίησαν ADV οἶδα οἶδα v1sria--- nosbj nosbj 9058369 7.7.22
cond 7.7.22 9058390 ἤν ποιήσῃς ἤν no-an
datposs 7.7.23 εἶναι εἰμί ἀνδρὶ
genagent 86 ἀνθρώπων OBJ ὑπὸ ὑπό ἀκούειν ἀκούω v--pna--- nosbj nosbj 9058414 7.7.23
cond 7.7.24 9058445 ἂν ὦσιν δύνανται no-an
cond 7.7.24 9058455 ἤν δέωνται δύνανται no-an
cond 7.7.24 9058470 ἤν βούλωνται γιγνώσκω no-an
cond 7.7.24 9058489 ἤν ὑπισχνῶνται διαπράττονται no-an
exw_objs 3154 ἃ OBJ ἔχεις ἔχω 9058574 7.7.27
datposs 7.7.28 γ

151 14 ἐντυγχάνω 0 8 1
152 14 ἀπορέω 0 2 3
153 14 κάθημαι 0 0 0
154 14 συσκευάζω 0 0 0
155 14 σκοπέω 5 0 0
156 14 ἀναπαύω 0 0 0
157 14 ἐφέπω 1 2 0
158 14 ἀποδιδράσκω 4 0 0
159 14 ἐκπλέω 0 0 0
160 13 τελευτάω 1 0 0
161 13 ἀποπέμπω 8 2 0
162 13 τρέφω 2 2 0
163 13 ἀποχωρέω 0 0 0
164 13 ἵημι 1 1 0
165 13 πυνθάνομαι 2 0 2
166 13 θνῄσκω 1 0 0
167 13 σπένδω 1 4 0
168 13 ἀπαλλάσσω 1 0 3
169 13 ἀναγκάζω 2 0 0
170 13 ἐννοέω 5 0 0
171 13 αὐλίζομαι 0 1 1
172 13 ἀριστάω 0 0 0
173 12 μεταπέμπω 8 0 0
174 12 ἀποδείκνυμι 4 5 0
175 12 ἐπιμελέομαι 0 0 5
176 12 ἐμβάλλω 3 1 0
177 12 ἥδομαι 0 3 0
178 12 ἀπολαμβάνω 6 3 0
179 12 προδίδωμι 6 1 0
180 12 συστρατεύω 0 3 0
181 12 κατακαίω 5 0 1
182 12 κρίνω 2 1 0
183 12 μιμνῄσκω 1 0 3
184 12 διαλέγω 2 4 0
185 12 παιανίζω 0 0 0
186 12 δηλόω 1 2 0
187 12 δειπνέω 0 0 0
188 12 ὄμνυμι 3 3 0
189 12 τολμάω 0 0 0
190 12 κλέπτω 3 0 1
191 12 θάπτω 5 0 0
192 11 ὑποπτεύω 1 0 0
193 11 ὑπάρχω 0 4 0
194 11 πολιορκέω 2 0 0
195 11 συγγίγνομαι 0 7 0
196 11 ἡσσάομαι 0 0 0
197 11 ἀπα

627 2 σατραπεύω 1 0 1
628 2 παρατείνω 1 0 0
629 2 πλήθω 0 0 0
630 2 προφαίνω 1 1 0
631 2 ἐνδύω 2 0 0
632 2 καταθεάομαι 1 0 0
633 2 πελάζω 0 1 0
634 2 συναντάω 0 1 0
635 2 ἐλελίζω 0 1 0
636 2 περιπίπτω 0 2 0
637 2 ἐπισφάζω 2 1 0
638 2 σπάω 2 0 0
639 2 φορέω 1 0 0
640 2 παιδεύω 0 0 0
641 2 ἐπιφέρω 0 0 0
642 2 συμπίπτω 0 0 0
643 2 ἀποκρύπτω 1 0 0
644 2 διαπέμπω 1 0 0
645 2 συντυγχάνω 1 2 0
646 2 ψιλόω 0 0 1
647 2 οἰάω 0 1 0
648 2 σπανίζω 0 0 2
649 2 λογίζομαι 1 0 0
650 2 ἐπισκοπέω 0 0 0
651 2 σχολάζω 0 0 0
652 2 ἀνερωτάω 1 0 0
653 2 μεθίστημι 1 0 0
654 2 βλακεύω 0 0 0
655 2 φροντίζω 0 0 0
656 2 παραθαρσύνω 0 0 0
657 2 ἁλίζω 0 1 0
658 2 κατατέμνω 1 0 0
659 2 διαιρέω 2 0 0
660 2 παραμελέω 0 1 0
661 2 ἐνοχλέω 0 1 0
662 2 ἄπειμι 0 0 0
663 2 παραμένω 0 0 0
664 2 ἀποστρέφω 0 0 0
665 2 ἀπειθέω 0 0 0
666 2 ἀπολιμπάνω 1 0 0
667 2 ἀπεχθάνομαι 0 2 0
668 2 ἀγάλλω 0 0 0
669 2 μηχανάομαι 1 0 0
670 2 μέμφομαι 2 0 0
671 2 λαγχάνω 1 0 1
672 2 λάμπω 0 0 0
673 2 ἀνεγείρω 0 0 0
674 2 προβαίνω 0 0 0
675 2 φαν

1067 1 ἔγκειμαι 0 0 0
1068 1 μύζω 0 0 0
1069 1 ἐπιχέω 1 0 0
1070 1 συμμανθάνω 0 0 0
1071 1 ἀντεμπλέω 1 0 0
1072 1 εὐθυμέω 0 0 0
1073 1 ἐπικύπτω 0 0 0
1074 1 ῥοφέω 0 0 0
1075 1 διακονέω 1 0 0
1076 1 περσίζω 0 0 0
1077 1 ἀνατρέφω 0 0 0
1078 1 κακόω 0 0 0
1079 1 περιειλέω 1 0 0
1080 1 εἰσφορέω 0 0 0
1081 1 ἔραμαι 0 0 1
1082 1 φθάζω 0 0 0
1083 1 μαστιγόω 0 0 0
1084 1 γέμω 0 0 0
1085 1 ἀποκάμνω 0 0 0
1086 1 συντρίβω 1 0 0
1087 1 λωφάω 0 0 0
1088 1 παραθέω 1 0 0
1089 1 διαγωνίζομαι 0 0 0
1090 1 ἐπικαταρριπτέω 1 0 0
1091 1 ἐπισπάω 1 0 0
1092 1 ἐμμένω 0 0 0
1093 1 διατρέφω 0 1 0
1094 1 φθείρω 1 0 0
1095 1 ζωγρέω 0 0 0
1096 1 παραβοηθέω 0 0 0
1097 1 δουλεύω 0 0 0
1098 1 ἐπιμαρτύρομαι 1 0 0
1099 1 συνεκκόπτω 1 0 0
1100 1 ἀναμείγνυμι 0 0 0
1101 1 ἀντιπαρατάσσομαι 1 0 0
1102 1 περισσεύω 0 0 0
1103 1 κατεσθίω 1 0 0
1104 1 ἀντιπαραθέω 0 0 0
1105 1 συνεφέπομαι 0 1 0
1106 1 ἀμόω 0 0 0
1107 1 διαχωρέω 0 1 0
1108 1 ἀναφρονέω 0 0 0
1109 1 συνδιαπράσσω 0 0 0
1110 1 παλαίω 0 0 0
1111 1 ἀνιάομαι 0 0 0
1112 

In [7]:
exw_objs

{'ὁπλίτης': 11,
 'φυλακή': 2,
 'ὅς': 18,
 'ἱππεύς': 13,
 'Ἀρκάς': 1,
 'Ταμώς': 1,
 'καπηλεῖον': 1,
 'γνώμη': 6,
 'ἐπιτήδειος': 22,
 'δύναμις': 12,
 'χρῆμα': 3,
 'ναῦς': 2,
 'στράτευμα': 6,
 'μυριάς': 1,
 'τριήρης': 11,
 'φρουρέω': 1,
 'πᾶς': 1,
 'Εὐφράτης': 2,
 'χιτών': 1,
 'ἀκρόπολις': 1,
 'ἱκανός': 1,
 'ὀλίγος': 3,
 'δεξιός': 1,
 'κέρας': 1,
 'κεφαλή': 2,
 'προμετωπίδιος': 1,
 'μάχαιρα': 1,
 'δρέπανον': 1,
 'μέσος': 4,
 'τάξις': 3,
 'χρύσεος': 1,
 'ὠτειλή': 1,
 'συνεργός': 1,
 'ὑπηρέτης': 1,
 'ὅπλον': 8,
 'ὑποζύγιον': 1,
 'στόλος': 1,
 'ἥλιος': 1,
 'ἡγεμών': 7,
 'δόρυ': 3,
 'βοήθεια': 1,
 'ἔρυμα': 1,
 'ποταμός': 2,
 'ἐλπίς': 1,
 'ἐγώ': 3,
 'πόρος': 1,
 'τιάρα': 1,
 'ὁ': 4,
 'ἔντερον': 1,
 'δίκη': 3,
 'οἷος': 2,
 'οὗτος': 14,
 'παιδικός': 1,
 'σῶμα': 1,
 'ψυχή': 1,
 'καιρός': 1,
 'ἀθυμία': 1,
 'πεῖρα': 1,
 'ὅσος': 2,
 'τίς': 2,
 'ὄχλος': 1,
 'θυγάτηρ': 1,
 'θώραξ': 2,
 'ἄκρον': 3,
 'ἀνήρ': 7,
 'γυμνής': 1,
 'ἀναγκαῖος': 1,
 'ὀπισθοφύλαξ': 3,
 'ἥμισυς': 2,
 'ἀσπίς': 2,
 'οὐδείς': 3,
 '

In [14]:
totdatobjs = 0

def verbdeps(curid):
    global totdatobjs
    curcit = citdict[curid]
    curpos,curlemma,curform,curhead,currel = getfeats(curid)
    headpos,headlemma,headform,headhead,headrel = getfeats(curhead)
    if(not curpos[0] == 'v'):
        #print('fail',curform)
        return
    
        
    #print('verb',curpos,curlemma,curform,curhead,currel)
    if(not curid in hasdeps):
        return
    for foo in hasdeps[curid]:
        deppos,deplemma,depform,dephead,deprel = getfeats(foo)
        if(deprel == 'OBJ' and re.search('[agd].$',deppos)):
            totdatobjs = 1 + totdatobjs
            print(totdatobjs,curcit,curid,curlemma,curform,'dep',deplemma,depform,deppos,deprel)

verbtot = 0
curverb = 'ἄχθομαι'
for formid in formdict:
    curpos,curlemma,curform,curhead,currel = getfeats(formid)
    if(not curlemma == curverb):
        continue
    verbtot = verbtot + 1
    verbdeps(formid)
print(verbtot,curverb)

    

1 3.2.20 9017390 ἄχθομαι ἄχθεσθε dep οὗτος τοῦτο a-s---na- OBJ
2 5.7.20 9038943 ἄχθομαι ἠχθόμεθά dep γίγνομαι γεγενημένοις v-prpend- OBJ
3 6.1.29 9041834 ἄχθομαι ἀχθόμενον dep σύ ὑμῖν p-p---cd- OBJ
4 7.6.10 9055895 ἄχθομαι ἄχθεσθαι dep οὐδείς οὐδὲν a-s---na- OBJ
5 7.7.20 9058326 ἄχθομαι ἠχθέσθης dep ἐγώ μοι p-s---cd- OBJ
10 ἄχθομαι


In [15]:
preplist

{'ἀπό': 112,
 'εἰς': 436,
 'πρός': 313,
 'ἐπί': 463,
 'ἀντί': 27,
 'παρά': 183,
 'ἐν': 440,
 'ἐκ': 249,
 'πλήν': 24,
 'κατά': 124,
 'ἀμφί': 50,
 'ὑπέρ': 41,
 'ὑπό': 114,
 'σύν': 169,
 'διά': 159,
 'περί': 123,
 'πρό': 19,
 'μετά': 87,
 'ἕως': 11,
 'ἄνευ': 18,
 'ἕνεκα': 37,
 'μά': 5,
 'μέχρι': 28,
 'χωρίς': 2,
 'πέραν': 5,
 'νή': 2,
 'μεταξύ': 3,
 'ἀνά': 12,
 'ἐντός': 7,
 'ἄχρι': 2,
 'ἔμπροσθεν': 4,
 'ἀντίον': 1,
 'πλησίον': 2,
 'ἐπέκεινα': 1,
 'δίχα': 1,
 'πέρα': 1,
 'ἐναντίον': 1}

In [3]:

for foo in subj2verbs:
    curdict = subj2verbs[foo]
    print('\n',foo)
    for boo in sorted(curdict,key=curdict.get,reverse=True):
        print(curdict[boo],boo)
        
print('datverbs',len(datverbs))

totdatvbs = 0

for foo in sorted(datverbs,key=datverbs.get,reverse=True):
    print(foo,datverbs[foo])
    totdatvbs = datverbs[foo] + totdatvbs
    
print('forms',totdatvbs)
        


 παῖς
1 γίγνομαι-e
1 πάρειμι-a
1 παιδεύω-e
1 θεάομαι-e
1 ἀγωνίζομαι-e
1 ποιέω-e

 Δαρεῖος
1 ἀσθενέω-a
1 τελευτάω-a

 πρέσβυς
1 τυγχάνω-a
1 προσλαμβάνω-a
1 ἔρχομαι-e

 ὅσος
7 εἰμί-a
1 ἀθροίζω-e
1 διαλέγω-e
1 πράσσω-p
1 συνέρχομαι-a
1 γίγνομαι-m
1 πολεμέω-p
1 ἀφικνέομαι-m
1 ἐμπίμπρημι-a
1 δύναμαι-p
1 κοιμάω-e
1 ἐσθίω-a
1 τυγχάνω-a

 Κῦρος
6 λέγω-a
4 ἀναβαίνω-a
4 δίδωμι-a
4 φημί-a
3 ἥδομαι-p
3 ἐξελαύνω-a
3 ἐρωτάω-a
3 τελευτάω-a
2 ἀποπέμπω-a
2 μένω-a
2 θεωρέω-a
2 μεταπέμπω-e
2 γίγνομαι-m
2 ποιέω-e
2 κελεύω-a
2 εἰμί-a
2 ἀποθνῄσκω-a
2 δοκέω-a
2 πέμπω-a
2 θνῄσκω-a
1 πολιορκέω-a
1 ἄγαμαι-p
1 ὁρμάω-e
1 ποιέω-a
1 ἀποδίδωμι-a
1 συγγίγνομαι-m
1 ἀποκτείνω-a
1 καλέω-a
1 φθάνω-a
1 ἀέκων-a
1 ὑπισχνέομαι-e
1 μεταπέμπω-m
1 οἴομαι-e
1 ἐάω-a
1 διώκω-a
1 ἐκκόπτω-a
1 δέω-e
1 οἶδα-m
1 διαβαίνω-a
1 ἐφίστημι-a
1 ἥκω-a
1 ἐπέρχομαι-a
1 συλλαμβάνω-a
1 ἄρχω-a
1 συμβουλεύω-e
1 προσελαύνω-a
1 παρέρχομαι-a
1 πορεύω-e
1 ἐνδύω-e
1 καθίστημι-e
1 βοάω-a
1 καταθεάομαι-e
1 ἐπιμελέομαι-e
1 ἐλαύνω-a
1 σπένδω-m
1 αἰσθάνομαι-

In [4]:
curdict = aobjnoms['στράτευμα']
i = 0
print(len(curdict))
for foo in sorted(curdict,key=curdict.get,reverse=True):
    i = i + 1
    print(i,foo,curdict[foo])
    


18
1 συλλέγω 4
2 ἔχω 4
3 ἀθροίζω 1
4 λαμβάνω 1
5 ἀποπέμπω 1
6 ἐπιδείκνυμι 1
7 εὑρίσκω 1
8 ἐπιλείπω 1
9 συναγείρω 1
10 διαγγέλλω 1
11 διαβαίνω 1
12 ἄγω 1
13 ἵστημι 1
14 καθίστημι 1
15 ἐφίστημι 1
16 ὠφελέω 1
17 συνάγω 1
18 προάγω 1


In [19]:
import re


tbvoc = {}
tottokens = 0

def addlemma(lem):
    if(lem in tbvoc):
        tbvoc[lem] = tbvoc[lem] + 1
    else:
        tbvoc[lem] = 1
        
def gettbvoc(fname):
    limlems = 69000
    i = 0
    global tottokens
    f = open(fname)
    for l in f:
        if(re.search('postag="u',l)):
            continue
        m = re.search('lemma="([^"]+)"',l)
        if(m):
            i = i + 1
            if(i>limlems):
                break
            curlem = re.sub('#[0-9]*','',m[1])
            addlemma(curlem)
            tottokens = tottokens + 1
        m = re.search("lemma='([^']+)'",l)
        if(m):
            i = i + 1
            if(i>limlems):
                break
            curlem = re.sub('#[0-9]*','',m[1])
            addlemma(curlem)
            tottokens = tottokens + 1
    
    f.close()
  
fname = '/Users/gcrane/github/proiel-treebank/greek-nt.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0011-002.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0003-001.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0059-030.xml'
fname = '/Users/gcrane/github/glaux-trees/public/xml/0032-006.xml'
fname = curtb

gettbvoc(fname)
print(len(tbvoc))

3687


In [20]:
import re
from greek_accentuation.characters import *
#exit

quizoutf = open('quizletvoc.tsv','w')
#do not run!
f = open('crosbyshaeffer2.0.xml')
#outf = open('cs2-2.xml','w')
prevpage = 0
prevpara = 0
curlesson = 0
sawvocab = 0
invocab = 0
gksent = 0
vocabitems = 0
totfreq = 0
csvocab = {}
csglosses = {}
for l in f:
    l = re.sub('@([0-9]+).*','<pb n="\g<1>"/>',l)
    l = re.sub('\n+$','',l)
    m = re.search('<pb n="([0-9]+)',l)
    if(m):
        curpage = int(m[1])
        if(not curpage == prevpage + 1):
            print('jumping from',prevpage,curpage)
        prevpage = curpage
    m = re.search('subtype="para"',l)
    if(m):
        invocab = 0
    if(re.search('^[0-9]+\.\s+.+[α-ω]',l)):
        gksent = gksent + 1
        #print(gksent,'gksent',l,end='')
    if(re.search('VOCAB',l)):
        sawvocab = 1
        invocab = 1
    m = re.search('<div type="textpart" subtype="lesson" n="([0-9]+)">',l)
    if(m):
        if( not sawvocab):
            print('missingvoc',curpage,curlesson)
       
        curlesson = m[1]
       
        sawvocab = 0
        l = re.sub('<div type="textpart" subtype="lesson">','<div type="textpart" subtype="lesson" n="'+str(curlesson)+'">',l)
    curlemma = ''
    m = re.search('^\s*<rs type="lemma" n="([^"]+)"',l)
    if(m):
        curlemma = m[1]
    elif(re.search('^\s*<rs type="lemma">([^<]+)',l)):
        m = re.search('^\s*<rs type="lemma">([^<]+)',l)
        curlemma = m[1]
    if(curlemma):
        vocabitems = vocabitems + 1
        if(not re.search('<rs',l)):
            l = re.sub('^([^ :,;]+)','<rs type="lemma">\g<1></rs>',l)
        curlemma = re.sub('-','',m[1])
        curlemma = strip_length(curlemma)
        csglosses[curlemma] = l
        if(not curlemma in csvocab):
            csvocab[curlemma] = []
        csvocab[curlemma].append(curlesson)
            #print(curlemma)
        if( not  curlemma in tbvoc):
            print('\nfailed',curlemma)
            curfreq = 0
        else:
            curfreq = tbvoc[curlemma]
            totfreq = totfreq + curfreq
        print(vocabitems, curlemma,curpage,curlesson,curfreq,totfreq,l,sep='\t')
        quizl = re.sub('<[^>]+>','',l)
        quizl = re.sub(':\s+','\t',quizl)
        print(vocabitems, curlemma,curpage,curlesson,curfreq,totfreq,quizl,sep='\t',file=quizoutf)
    #print(l,end='',file=outf)

quizoutf.close()

missingvoc 1 0
1	ἀδελφός	2	1	20	20	<rs type="lemma">ἀδελφός</rs>, ἀδελφοῦ, ὁ: brother.  PHILADELPHIA.

failed ἔχει
2	ἔχει	2	1	0	20	<rs type="lemma">ἔχει</rs>: he, she, or it has,

failed ἔχουσι
3	ἔχουσι	2	1	0	20	<rs type="lemma">ἔχουσι</rs>: they have.

failed ἦν
4	ἦν	2	1	0	20	<rs type="lemma">ἦν</rs>: he, she, it, or there was.

failed ἦσαν
5	ἦσαν	2	1	0	20	<rs type="lemma">ἦσαν</rs>: they or there were.

failed παύει
6	παύει	2	1	0	20	<rs type="lemma">παύει</rs>: he, she, or it stops (transitive).

failed παύουσι
7	παύουσι	2	1	0	20	<rs type="lemma">παύουσι</rs>: they stop.

failed πέμπει
8	πέμπει	2	1	0	20	<rs type="lemma">πέμπει</rs>: he, she, or it sends (transitive).

failed πέμπουσι
9	πέμπουσι	2	1	0	20	<rs type="lemma">πέμπουσι</rs>: they send.
10	ποταμός	2	1	99	119	<rs type="lemma">ποταμός</rs>, ποταμοῦ, ὁ: river, HIPPOPOTAMUS.
11	στρατηγός	2	1	151	270	<rs type="lemma">στρατηγός</rs>, στρατηγοῦ, ὁ: general. STRATEGY.
12	ἄνθρωπος	5	2	75	345	<rs type="lemma">ἄνθρωπος</rs>, ἀνθρώπου, 

298	ἀδικέω	106	33	38	32232	<rs type="lemma">ἀ-δικέω</rs>, See § 205,  Cf. mopeta, wopetopar,
299	ἀναγκαῖος	106	33	6	32238	<rs type="lemma">ἀναγκαῖος</rs>: necessary. Cf.  ἀνάγκη.
300	στρατεύω	106	33	24	32262	<rs type="lemma">στρατεύω</rs>, στρατεύσω, ἐστράτευσα : make campaign. Usually mid.
301	δίκη	106	33	24	32286	<rs type="lemma">δίκη</rs>: justice. Cf. στρατιά.
302	ἐπιθυμέω	106	33	20	32306	<rs type="lemma">ἐπιθυμέω</rs>, ἐπιθυμ-ήσω, ἐπ-εθύμησα: desire, with G, Cf. θυμός.  
303	τάσσω	106	33	34	32340	<rs type="lemma" n="τάσσω">τάττω</rs>, τάξω, ἔταξα: order, arrange, station. TAXIDERMIST, TACTICS.
304	ἤ	106	33	229	32569	<rs type="lemma">ἤ</rs>, conj. : or, than. 
305	οἶνος	106	33	26	32595	<rs type="lemma">οἶνος</rs>: wine. 
306	τρέφω	106	33	13	32608	<rs type="lemma">τρέφω</rs>, θρέψω, ἔθρεψα: nourish, support, rear, keep (of animals). ATROPHY.
307	πάλιν	106	33	69	32677	<rs type="lemma">πάλιν</rs>, adv. : again, back,  PALINODE. </p></div>
308	ἀποθνῄσκω	109	34	49	32726	<rs type="lemma"

577	μέχρι	227	67	27	45502	<rs type="lemma">μέχρι</rs>, conj.: until, while. Also  used as a prep. with G.: up to.
578	τελευτάω	227	67	13	45515	<rs type="lemma">τελευτάω</rs>, τελευτήσω, ἐτελεύτησα, τετελεύτηκα, τετελεύτημαι, ἐτελευτήθην:  end, finish, die.
579	μηχανάομαι	227	67	2	45517	<rs type="lemma">μηχανάομαι</rs>, μηχανήσομαι, ἐμηχανησάμην : contrive. 
580	τιμάω	227	67	18	45535	<rs type="lemma">τῑμάω</rs>, τῑμήσω, ἐτίμησα, τετίμηκα, τετίμημαι, ἐτῑμήθην: honor, reward,
581	βία	230	68	11	45546	<rs type="lemma">βίᾱ</rs>, βίᾱς, ἡ: force, violence. Cf. δύναμις: force, power. 
582	δαπανάω	230	68	6	45552	<rs type="lemma">δαπανάω</rs>, δαπανήσω, ἐδαπάνησα, δεδαπάνηκα, δεδαπάνημαι, ἐδαπανήθην : spend, waste.
583	νικάω	230	68	42	45594	<rs type="lemma">νῑκάω</rs>, νῑκήσω, ἐνίκησα, νενίκηκα,  νενίκημαι, ἐνῑκήθην: conquer,  win.
584	νίκη	230	68	5	45599	<rs type="lemma">νίκη</rs>, νίκης, ἡ: victory. EUNICE.
585	οἶδα	230	68	100	45699	<rs type="lemma">οἶδα</rs>, εἴσομαι: know.
586	ἔξω	230	68	37	4

In [17]:
def prepuse(curprep):
    curacc = 0
    curgen = 0
    curdat = 0
    if( curprep in prep_acc ):
        curacc = prep_acc[curprep]
    if( curprep in prep_gen ):
        curgen = prep_gen[curprep]
    if( curprep in prep_dat ):
        curdat = prep_dat[curprep]
    print(curprep,preplist[curprep],curgen,curdat,curacc,sep='\t')

print('curprep','total','gen.','dat.','acc.',sep='\t')
for foo in sorted(preplist,key=preplist.get,reverse=True):
    prepuse(foo)              

curprep	total	gen.	dat.	acc.
ἐπί	463	70	121	267
ἐν	440	1	435	0
εἰς	436	0	0	409
πρός	313	25	16	264
ἐκ	249	245	0	0
παρά	183	72	37	73
σύν	169	1	168	0
διά	159	92	0	47
κατά	124	9	0	115
περί	123	71	4	45
ὑπό	114	97	8	7
ἀπό	112	112	0	0
μετά	87	28	0	59
ἀμφί	50	1	0	44
ὑπέρ	41	38	0	2
ἕνεκα	37	35	0	1
μέχρι	28	14	0	1
ἀντί	27	26	0	0
πλήν	24	15	0	0
πρό	19	19	0	0
ἄνευ	18	18	0	0
ἀνά	12	0	0	11
ἕως	11	0	0	0
ἐντός	7	7	0	0
μά	5	0	0	5
πέραν	5	4	1	0
ἔμπροσθεν	4	2	0	1
μεταξύ	3	3	0	0
χωρίς	2	1	0	1
νή	2	0	0	2
ἄχρι	2	0	0	0
πλησίον	2	2	0	0
ἀντίον	1	0	0	1
ἐπέκεινα	1	1	0	0
δίχα	1	0	0	0
πέρα	1	1	0	0
ἐναντίον	1	1	0	0


In [6]:
len(depsdict)
i = 0
for foo in depsdict:
    i = i + 1
    print(i,foo,depsdict[foo])
    if(i>20):
        break

i = 0
for foo in formdict:

    headpos = get_postag(formdict[foo])
    curlemma = get_lemma(formdict[foo])
    if(not is_decl1(curlemma)):
        continue
    m = re.search('(.).(.---m.-)',headpos)
    if(m and m[1] == 'n'):
        matchpos = m[2]
        if(not foo in depsdict):
            continue
        matchart = has_matchingarticle(foo,matchpos)
        if(not matchart):
            continue
        for boo in depsdict[foo]:
            if(not re.search('ATR',get_relation(formdict[boo]))):
                continue
            deppos = get_postag(formdict[boo])
            if(not is_decl1(get_lemma(formdict[boo]))):
                continue
            if(re.search('^a',deppos) and re.search(matchpos,deppos)):
                print()
                print(i,citdict[foo])
                matchart2 = has_matchingarticle(boo,matchpos)
                
                if(matchart<boo and boo < foo):
                    i = i + 1
                    if(i>20):
                        break
                    print('attrib1')
                    print(formdict[matchart])
                    print(formdict[boo])
                    print(formdict[foo])
                else:
                    print(formdict[matchart])
                    print(formdict[foo])
                    print(formdict[boo])
    


1 8995997 ['8995993', '8995998']
2 8995995 ['8995994']
3 8995993 ['8995995', '8995999', '8996003', '8996007']
4 8995996 ['8995997', '8996001', '8996002', '8996005']
5 8996002 ['8996000', '8996006']
6 8996006 ['8996004']
7 8996018 ['8996008', '8996009', '8996022']
8 8996008 ['8996010', '8996017', '8996023']
9 8996010 ['8996011', '8996013']
10 8996013 ['8996012', '8996014']
11 8996016 ['8996015']
12 8996014 ['8996016']
13 8996020 ['8996019']
14 8996022 ['8996020', '8996021']
15 8996027 ['8996024']
16 8996029 ['8996025', '8996026', '8996027', '8996028']
17 8996024 ['8996030']
18 8996033 ['8996031', '8996032', '8996034', '8996046']
19 8996036 ['8996035', '8996040']
20 8996034 ['8996036']
21 8996039 ['8996037']

0 1.1.6
    <word id="8996228" form="τοῖς" lemma="ὁ" postag="l-p---md-" head="8996229" relation="ATR"/>
    <word id="8996229" form="φρουράρχοις" lemma="φρούραρχος" postag="n-p---md-" head="8996227" relation="OBJ"/>
    <word id="8996230" form="ἑκάστοις" lemma="ἕκαστος" postag="a-p-

In [6]:
vparads = ['pia','iia','aia','fia','ria','lia','tia',
           'psa','asa','fsa','rsa','tsa',
           'poa','aoa','foa','roa','toa',
           'pma','ama','fma','rma','tma',
           'ppa','apa','fpa','rpa','tpa',
           'pna','ana','fna','rna','tna',
            
            'pim','iim','aim','fim','rim','lim','tim',
           'psm','asm','fsm','rsm','tsm',
           'pom','aom','fom','rom','tom',
           'pmm','amm','fmm','rmm','tmm',
           'ppm','apm','fpm','rpm','tpm',
           'pnm','anm','fnm','rnm','tnm',
            
            'pip','iip','aip','fip','rip','lip','tip',
           'psp','asp','fsp','rsp','tsp',
           'pop','aop','fop','rop','top',
           'pmp','amp','fmp','rmp','tmp',
           'ppp','app','fpp','rpp','tpp',
           'pnp','anp','fnp','rnp','tnp',
            
            'pie','iie','aie','fie','rie','lie','tie',
           'pse','ase','fse','rse','tse',
           'poe','aoe','foe','roe','toe',
           'pme','ame','fme','rme','tme',
           'ppe','ape','fpe','rpe','tpe',
           'pne','ane','fne','rne','tne',
           ]

tensenames = {
 "p" : "present",
 "i" : "imperfect",
 "r" : "perfect",
 "s" : "resultative",
 "a" : "aorist",
 "u" : "past",
 "l" : "pluperfect",
 "f" : "future",
 "t" : "future perfect",
 "x" : "uncertain tense"
}

moodnames = {
 "i" : "indicative",
 "s" : "subjunctive",
 "m" : "imperative",
 "o" : "optative",
 "n" : "infinitive",
 "p" : "participle",
 "d" : "gerund"
}

voicenames = {
    
 "a" : "active",
 "m" : "middle",
 "p" : "passive",
 "e" : "middle or passive",
 "x" : "unspecified",
}

totvbs  = 0
vparadcnt = {}
def countvparads(curparad):
    global totvbs
    f = open(curtb)
    vpat = 'postag=.v..(...)'
    
    for foo in vparads:
        vparadcnt[foo] = 0
    for l in f:
        m = re.search(vpat,l)
        if(m):
            curv = m[1]
            if(not curv in vparadcnt):
                print(l)
            else:
                vparadcnt[curv] = vparadcnt[curv] + 1
                totvbs = totvbs + 1
        
    
    f.close()

countvparads('pia')
i = 0
sofar = 0
for foo in sorted(vparadcnt,key=vparadcnt.get,reverse=True):
    i = i + 1
    sofar = sofar + vparadcnt[foo]
    print(i,foo,vparadcnt[foo],vparadcnt[foo]/totvbs,sofar/totvbs)

1 iia 1890 0.14210526315789473 0.14210526315789473
2 ppa 1346 0.10120300751879699 0.24330827067669172
3 pia 1198 0.09007518796992481 0.33338345864661656
4 pna 1112 0.08360902255639098 0.4169924812030075
5 aia 1084 0.08150375939849624 0.49849624060150377
6 apa 758 0.05699248120300752 0.5554887218045113
7 iie 699 0.05255639097744361 0.6080451127819548
8 ppe 495 0.037218045112781956 0.6452631578947369
9 ana 440 0.03308270676691729 0.6783458646616541
10 pie 431 0.03240601503759399 0.7107518796992481
11 pne 400 0.03007518796992481 0.740827067669173
12 poa 362 0.027218045112781954 0.7680451127819549
13 aim 346 0.026015037593984963 0.7940601503759398
14 fim 204 0.015338345864661655 0.8093984962406015
15 rpe 194 0.014586466165413535 0.823984962406015
16 apm 177 0.01330827067669173 0.8372932330827068
17 psa 166 0.012481203007518798 0.8497744360902255
18 aip 152 0.011428571428571429 0.861203007518797
19 fia 146 0.010977443609022556 0.8721804511278195
20 poe 145 0.010902255639097745 0.88308270676

In [21]:

stoplist = ['ὁ','καί','οὐκ', 'δέ','εἰμί','ἄπειμι']

def searchvoc(oldlemlist,searchlemlist,posfilts):
    oldlist = []
    #searchlem = searchlemlist[2]
    i = 0
    outf = open('currquest.tsv','w')
    #if(searchlem in greekforms and not searchlem in greeklems):
    #if(searchlem in greekforms and not intersection(searchlemlist,  greeklems)):
    if(1 and not intersection(searchlemlist,  greeklems)):
        print('sawform',searchlem)
        for foo in sorted(greeksentlens,key=greeksentlens.get):
            if(searchlem in greeksentforms[foo]):
                i = i + 1
                ind = greeksentforms[foo].index(searchlem)
                print('hit',i,engsentcits[foo],len(greeksentlems[foo]),foo,greeksentforms[foo][ind],greeksentpostags[foo][ind],greeksentrels[foo][ind])
                print(' '.join(greeksentforms[foo]))
                print(engsents[foo])
                print(' '.join(greeksentlems[foo]))
                print()
    else:
        for foo in sorted(greeksentlens,key=greeksentlens.get):
            #if(searchlem in greeksentlems[foo]):
            interl = intersection(searchlemlist+oldlemlist,  greeksentlems[foo])
            interl1 = intersection(oldlemlist,  greeksentlems[foo])
            if(interl and posfilts):
                workinterl = []
                if(len(posfilts)>1):
                    possearch = '('+'|'.join(posfilts) + ')'
                else:
                    possearch = posfilts[0]
                for cursearchlem in interl:
                    ind = greeksentlems[foo].index(cursearchlem)
                    if(re.search(possearch,greeksentpostags[foo][ind])):
                        workinterl.append(cursearchlem)
                interl = workinterl
            if(interl1 and 0):
                for goo in greeksentforms[foo]:
                    oldlist.append(goo)
                #newlist = greeksentforms[foo]
                i = i + 1
                for cursearchlem in interl1:
                    #if(cursearchlem in stoplist):
                     #   continue

                    ind = greeksentlems[foo].index(cursearchlem)
                    #newlist = greeksentforms[foo]
                    #print('old',oldlemlist,oldlemlist[ind])
            if(interl):
                newlist = []
                for goo in greeksentforms[foo]:
                    newlist.append(goo)
                #newlist = greeksentforms[foo]
                i = i + 1
                print(i,interl)
                answers = []
                sentrels = []
                for cursearchlem in interl:
                    if(cursearchlem in stoplist):
                        continue

                    ind = greeksentlems[foo].index(cursearchlem)
                    #newlist = greeksentforms[foo]
                    print('new',newlist,newlist[ind])
                    answers.append(newlist[ind]) 
                    sentrels.append(greeksentrels[foo][ind])
                    hiddenform = newlist[ind]
                    #newlist[ind] = '[______]'
                #just book 1 for now
                if(not re.search('^1',engsentcits[foo])):
                    continue
                print(i,foo,engsentcits[foo],end='\t',sep='\t',file=outf)
#print(i,engsentcits[foo],len(greeksentlems[foo]),foo,greeksentforms[foo][ind],greeksentpostags[foo][ind],greeksentrels[foo][ind])
                print(' '.join(newlist),' '.join(oldlist),engsents[foo],end='\t',sep='\t',file=outf)
                #print(engsents[foo],file=outf)
                #print(' '.join(greeksentlems[foo]))
                #print(' '.join(greeksentlems[foo]))
                print(interl,len(answers),answers,sentrels,file=outf,sep='\t')
                print()
    print('done' )      
    outf.close()       

def totvoclist(startn,lnum):
    vblist=[]
    nomlist = []
    oldlist = []
    for i in range(0,startn):
        curchap = str(i)
        
        for boo in csvocab:
            if(boo in stoplist):
                continue
            if( curchap in csvocab[boo]):
                #print(curchap,'got',boo,csvocab[boo])
                oldlist.append(boo)
        
    for i in range(startn,lnum+1):
        curchap = str(i)
        
        for boo in csvocab:
            if(boo in stoplist):
                continue
            if( curchap in csvocab[boo]):
                #print(curchap,'got',boo,csvocab[boo])
                if(re.search('(ω|όμαι)$',boo)):
                    vblist.append(boo)
                else:
                    nomlist.append(boo)
    return(oldlist,vblist,nomlist)

#vblist,nomlist = totvoclist(10)
#vblist12,nomlist12 = totvoclist(12)
#vblist13,nomlist13 = totvoclist(13)
#vblist14,nomlist14 = totvoclist(14)
#vblist15,nomlist15 = totvoclist(15)
#vblist16,nomlist16 = totvoclist(16)
#vblist17,nomlist17 = totvoclist(17)
#vblist22,nomlist22 = totvoclist(1,22)
oldlemmas,vblist24,nomlist24 = totvoclist(24,24)
#vblist25,nomlist25 = totvoclist(25,25)
#print(vblist)
#searchvoc(nomlist,[])
#searchvoc(['ὅς'],[])
#vblist12_16 = vblist12 + vblist13 + vblist14 + vblist15 + vblist16
#nomlist12_16 = nomlist12 + nomlist13 + nomlist14 + nomlist15 + nomlist16
#searchvoc(vblist12_16,['iia','pia','pna','fia','fna','aia','ana'])
#nomlist17_18 = nomlist17 + nomlist18
#searchvoc(nomlist18,[])
#searchvoc(['πᾶς'],[])
#searchvoc(vblist24,['iia','pia','pna','fia','fna','aia','ana',
                   # 'ppa','apa','fpa','iie','iip','pie','pim','pip'])
#oldlemmas,vblist25,nomlist25 = totvoclist(25,25)
#searchvoc(oldlemmas,vblist25+nomlist25,[])
oldlemmas,vblist4,nomlist4 = totvoclist(1,10)
#searchvoc(oldlemmas,vblist26+nomlist26,[])
#searchvoc(['ἀπό','ἐκ','ἐν','παρά','εἰς','διά','ἐπί','σύν','κατά','πρός'],[])
#searchvoc(vblist4,['iia','pia','pna','fia','fna'])
searchvoc([],vblist4,['pia'])
searchvoc([],nomlist4,[])


1 ['ἔχω']
new ['οὕτω', 'δὲ', 'ἔχει', '·'] ἔχει
2 ['ἔχω']
new ['ἔχω', 'δὲ', 'καὶ', 'ἡγεμόνας', '·'] ἔχω
3 ['ἔχω']
new ['ἔχομεν', 'γὰρ', 'τὰ', 'ἐκείνων', '·'] ἔχομεν
4 ['ἔχω']
new ['πλοῖα', 'γὰρ', 'οὐκ', 'ἔχομεν', '·'] ἔχομεν
5 ['ἔχω']
new ['νῦν', 'δὲ', 'οὕτως', 'ἔχει', '·'] ἔχει
6 ['ἔχω']
new ['πλοῖα', 'δὲ', 'ἡμεῖς', 'οὐκ', 'ἔχομεν', '.'] ἔχομεν
7 ['ἔχω']
new ['ἀλλ’', 'ἐγὼ', 'ἔχω', 'δύο', 'ἄνδρας', '.'] ἔχω
8 ['ἔχω']
new ['ἀπιόντας', 'δειπνεῖν', 'ὅ', 'τι', 'τις', 'ἔχει', '·'] ἔχει
9 ['ἔχω']
new ['ὁ', 'δὲ', 'τοὺς', 'ἡμετέρους', 'πόνους', 'ἔχει', '·'] ἔχει
10 ['ἔχω']
new ['ἔχω', 'γὰρ', 'καὶ', 'αὐτὸς', 'αὐτῷ', 'μαρτυρῆσαι', '.'] ἔχω
11 ['νομίζω']
new ['ὅτι', 'μέντοι', 'ἀδικεῖσθαι', 'νομίζει', 'ὑφ’', 'ἡμῶν', 'οἶδα', '·'] νομίζει

12 ['ἐθέλω']
new ['καὶ', 'προσελθόντες', 'εἰς', 'ἐπήκοον', 'ἠρώτων', 'τί', 'θέλει', '.'] θέλει
13 ['ἔχω']
new ['ἔχω', 'γὰρ', 'τριήρεις', 'ὥστε', 'ἑλεῖν', 'τὸ', 'ἐκείνων', 'πλοῖον', '·'] ἔχω

14 ['ἔχω']
new ['ταύτην', ',', 'ἔφη', ',', 'τὴν', 'γνώμην', 'ἔχω', 'ἔγωγε'

303 ['ἵππος']
new ['Ξενοφῶν', 'δὲ', 'παρελαύνων', 'ἐπὶ', 'τοῦ', 'ἵππου', 'παρεκελεύετο', '·'] ἵππου
304 ['διά']
new ['τούτων', 'δ’', 'οὐδέν’', 'ἀπονοστῆσαι', 'διὰ', 'τὴν', 'δυσχωρίαν', '.'] διὰ
305 ['ἐπιτήδειος']
new ['τὰ', 'μέντοι', 'ἐπιτήδεια', 'ὅτῳ', 'τις', 'ἐπιτυγχάνοι', 'ἐλάμβανεν', '·'] ἐπιτήδεια
306 ['ἐκ', 'γάρ']
new ['ἐξ', 'ἀπροσδοκήτου', 'γὰρ', 'αὐτοῖς', 'ἐπέπεσε', 'τὸ', 'Ἑλληνικόν', '.'] ἐξ
new ['ἐξ', 'ἀπροσδοκήτου', 'γὰρ', 'αὐτοῖς', 'ἐπέπεσε', 'τὸ', 'Ἑλληνικόν', '.'] γὰρ
307 ['μακρός']
new ['ὅπλα', 'δ’', 'εἶχον', 'γέρρα', 'μακρὰ', 'καὶ', 'λόγχας', '.'] μακρὰ
308 ['ποταμός']
new ['εἰ', 'δὲ', 'μή', ',', 'ἥρπαζεν', 'ὁ', 'ποταμός', '·'] ποταμός
309 ['σπονδή']
new ['ἀκούσας', 'δὲ', 'καὶ', 'ὁ', 'Χειρίσοφος', 'σπονδὰς', 'ἐποίει', '.'] σπονδὰς
310 ['ἐν', 'γάρ']
new ['πολλαὶ', 'γὰρ', 'ἦσαν', 'ἑταῖραι', 'ἐν', 'τῷ', 'στρατεύματι', '.'] ἐν
new ['πολλαὶ', 'γὰρ', 'ἦσαν', 'ἑταῖραι', 'ἐν', 'τῷ', 'στρατεύματι', '.'] γὰρ
311 ['κώμη', 'ποταμός']
new ['κῶμαι', 'δὲ', 'πολλαὶ', 'περὶ', 'τὸν', 'πο

new ['καὶ', 'κραυγῆς', 'γενομένης', 'εἰς', 'τὰ', 'ὅπλα', 'πάντες', 'ἔδραμον', 'οἱ', 'Ἕλληνες', '·'] εἰς
691 ['μέν', 'οὐ']
new ['οἱ', 'μὲν', 'δὴ', 'ἄλλοι', 'πάντες', 'ἐξῇσαν', ',', 'Νέων', 'δὲ', 'οὔ', '·'] μὲν
new ['οἱ', 'μὲν', 'δὴ', 'ἄλλοι', 'πάντες', 'ἐξῇσαν', ',', 'Νέων', 'δὲ', 'οὔ', '·'] οὔ
692 ['καλός', 'γάρ']
new ['ἐδόκει', 'γὰρ', 'κάλλιστον', 'εἶναι', 'τοῦτον', 'φύλακα', 'καταλιπεῖν', 'τῶν', 'ἐπὶ', 'στρατοπέδου', '.'] κάλλιστον
new ['ἐδόκει', 'γὰρ', 'κάλλιστον', 'εἶναι', 'τοῦτον', 'φύλακα', 'καταλιπεῖν', 'τῶν', 'ἐπὶ', 'στρατοπέδου', '.'] γὰρ
693 ['πολέμιος', 'καλός']
new ['οἱ', 'δὲ', 'πολέμιοι', 'ὑπέμενον', ',', 'νομίζοντες', 'καλὸν', 'ἔχειν', 'τὸ', 'χωρίον', '.'] πολέμιοι
new ['οἱ', 'δὲ', 'πολέμιοι', 'ὑπέμενον', ',', 'νομίζοντες', 'καλὸν', 'ἔχειν', 'τὸ', 'χωρίον', '.'] καλὸν
694 ['πολέμιος', 'γάρ']
new ['τὸ', 'γὰρ', 'ἱππικὸν', 'φόβον', 'παρεῖχε', 'τὸ', 'τῶν', 'πολεμίων', 'πολὺ', 'ὄν', '.'] πολεμίων
new ['τὸ', 'γὰρ', 'ἱππικὸν', 'φόβον', 'παρεῖχε', 'τὸ', 'τῶν', 'πολεμίων', 'πολὺ',

new ['τούτων', 'δὲ', 'τοιούτων', 'ὄντων', 'τίς', 'οὕτω', 'μαίνεται', 'ὅστις', 'οὐ', 'βούλεται', 'σοὶ', 'φίλος', 'εἶναι', ';'] οὐ
1032 ['ἐκ']
new ['καὶ', 'ᾤετο', 'κτήσεσθαι', 'ἐκ', 'τούτων', 'ὄνομα', 'μέγα', 'καὶ', 'δύναμιν', 'μεγάλην', 'καὶ', 'χρήματα', 'πολλά', '·'] ἐκ
1033 ['φίλος']
new ['φίλος', 'τε', 'ἐβούλετο', 'εἶναι', 'τοῖς', 'μέγιστα', 'δυναμένοις', ',', 'ἵνα', 'ἀδικῶν', 'μὴ', 'διδοίη', 'δίκην', '.'] φίλος
1034 ['ἐπιστολή']
new ['ὁ', 'μέντοι', 'Ξενοφῶν', 'ἀναγνοὺς', 'τὴν', 'ἐπιστολὴν', 'ἀνακοινοῦται', 'Σωκράτει', 'τῷ', 'Ἀθηναίῳ', 'περὶ', 'τῆς', 'πορείας', '.'] ἐπιστολὴν
1035 ['πολέμιος', 'οὐ', 'γάρ']
new ['οὐ', 'γὰρ', 'ἔγωγ’', 'ἔτι', 'πρεσβύτερος', 'ἔσομαι', ',', 'ἐὰν', 'τήμερον', 'προδῶ', 'ἐμαυτὸν', 'τοῖς', 'πολεμίοις', '.'] πολεμίοις
new ['οὐ', 'γὰρ', 'ἔγωγ’', 'ἔτι', 'πρεσβύτερος', 'ἔσομαι', ',', 'ἐὰν', 'τήμερον', 'προδῶ', 'ἐμαυτὸν', 'τοῖς', 'πολεμίοις', '.'] οὐ
new ['οὐ', 'γὰρ', 'ἔγωγ’', 'ἔτι', 'πρεσβύτερος', 'ἔσομαι', ',', 'ἐὰν', 'τήμερον', 'προδῶ', 'ἐμαυτὸν', 'τοῖς', 'πολε

1364 ['ἐν']
new ['παρῆν', 'δὲ', 'καὶ', 'Σεύθης', 'βουλόμενος', 'εἰδέναι', 'τί', 'πραχθήσεται', ',', 'καὶ', 'ἐν', 'ἐπηκόῳ', 'εἱστήκει', 'ἔχων', 'ἑρμηνέα', '·'] ἐν
1365 ['ἀγαθός']
new ['ὥστε', ',', 'ὡς', 'ἐγὼ', 'οἶμαι', ',', 'οὐδὲ', 'τούτοις', 'δοκεῖτε', 'βελτίονες', 'εἶναι', 'τοιοῦτοι', 'ὄντες', 'περὶ', 'ἐμέ', '.'] βελτίονες
1366 ['κώμη', 'ἐν']
new ['ὁρῶν', 'οὖν', 'ὁ', 'Μηδοσάδης', 'δαπανώμενα', 'τὰ', 'ἑαυτοῦ', 'ἐν', 'ταῖς', 'κώμαις', 'ὑπὸ', 'τῶν', 'Ἑλλήνων', 'χαλεπῶς', 'ἔφερε', '·'] κώμαις
new ['ὁρῶν', 'οὖν', 'ὁ', 'Μηδοσάδης', 'δαπανώμενα', 'τὰ', 'ἑαυτοῦ', 'ἐν', 'ταῖς', 'κώμαις', 'ὑπὸ', 'τῶν', 'Ἑλλήνων', 'χαλεπῶς', 'ἔφερε', '·'] ἐν
1367 ['μέν']
new ['ἐγὼ', 'μέν', ',', 'ὦ', 'Μηδόσαδες', ',', 'κατὰ', 'τῆς', 'γῆς', 'καταδύομαι', 'ὑπὸ', 'τῆς', 'αἰσχύνης', 'ἀκούων', 'ταῦτα', '.'] μέν
1368 ['καλός', 'οὐ']
new ['ὥστε', 'οὐχ', 'οἷόν', 'τέ', 'σοι', 'λανθάνειν', 'οὔτε', 'ἤν', 'τι', 'καλὸν', 'οὔτε', 'ἤν', 'τι', 'αἰσχρὸν', 'ποιήσῃς', '.'] καλὸν
new ['ὥστε', 'οὐχ', 'οἷόν', 'τέ', 'σοι', 'λανθάνειν',

1675 ['ἀλλά', 'μέν']
new ['ἀλλ’', ',', 'ὦ', 'ἄνδρες', ',', 'τοῦτο', 'μὲν', 'ἴστε', ',', 'ὅτι', 'οὐδ’', 'ἂν', 'ἔγωγε', 'ἐστασίαζον', ',', 'εἰ', 'ἄλλον', 'εἵλεσθε', '·'] ἀλλ’
new ['ἀλλ’', ',', 'ὦ', 'ἄνδρες', ',', 'τοῦτο', 'μὲν', 'ἴστε', ',', 'ὅτι', 'οὐδ’', 'ἂν', 'ἔγωγε', 'ἐστασίαζον', ',', 'εἰ', 'ἄλλον', 'εἵλεσθε', '·'] μὲν
1676 ['ἀπό', 'ἐκ', 'ἀλλά', 'οὐ']
new ['ὥστε', 'κινηθῆναι', 'οὐκ', 'ἐδύναντο', 'ἐκ', 'τοῦ', 'χωρίου', ',', 'ἀλλὰ', 'τελευτῶντες', 'καὶ', 'ἀπὸ', 'τοῦ', 'ὕδατος', 'εἶργον', 'αὐτοὺς', 'οἱ', 'Θρᾷκες', '.'] ἀπὸ
new ['ὥστε', 'κινηθῆναι', 'οὐκ', 'ἐδύναντο', 'ἐκ', 'τοῦ', 'χωρίου', ',', 'ἀλλὰ', 'τελευτῶντες', 'καὶ', 'ἀπὸ', 'τοῦ', 'ὕδατος', 'εἶργον', 'αὐτοὺς', 'οἱ', 'Θρᾷκες', '.'] ἐκ
new ['ὥστε', 'κινηθῆναι', 'οὐκ', 'ἐδύναντο', 'ἐκ', 'τοῦ', 'χωρίου', ',', 'ἀλλὰ', 'τελευτῶντες', 'καὶ', 'ἀπὸ', 'τοῦ', 'ὕδατος', 'εἶργον', 'αὐτοὺς', 'οἱ', 'Θρᾷκες', '.'] ἀλλὰ
new ['ὥστε', 'κινηθῆναι', 'οὐκ', 'ἐδύναντο', 'ἐκ', 'τοῦ', 'χωρίου', ',', 'ἀλλὰ', 'τελευτῶντες', 'καὶ', 'ἀπὸ', 'τοῦ', 'ὕδατος', 

new ['ἄνευ', 'γὰρ', 'ἀρχόντων', 'οὐδὲν', 'ἂν', 'οὔτε', 'καλὸν', 'οὔτε', 'ἀγαθὸν', 'γένοιτο', 'ὡς', 'μὲν', 'συνελόντι', 'εἰπεῖν', 'οὐδαμοῦ', ',', 'ἐν', 'δὲ', 'δὴ', 'τοῖς', 'πολεμικοῖς', 'παντάπασιν', '.'] γὰρ
1951 ['ἀγαθός', 'ἐν', 'γάρ']
new ['ἃ', 'καὶ', 'ὑμᾶς', 'δεῖ', 'νῦν', 'καταμαθόντας', '(', 'ἐν', 'τοιούτῳ', 'γὰρ', 'καιρῷ', 'ἐσμεν', ')', 'αὐτούς', 'τε', 'ἄνδρας', 'ἀγαθοὺς', 'εἶναι', 'καὶ', 'τοὺς', 'ἄλλους', 'παρακαλεῖν', '.'] ἀγαθοὺς
new ['ἃ', 'καὶ', 'ὑμᾶς', 'δεῖ', 'νῦν', 'καταμαθόντας', '(', 'ἐν', 'τοιούτῳ', 'γὰρ', 'καιρῷ', 'ἐσμεν', ')', 'αὐτούς', 'τε', 'ἄνδρας', 'ἀγαθοὺς', 'εἶναι', 'καὶ', 'τοὺς', 'ἄλλους', 'παρακαλεῖν', '.'] ἐν
new ['ἃ', 'καὶ', 'ὑμᾶς', 'δεῖ', 'νῦν', 'καταμαθόντας', '(', 'ἐν', 'τοιούτῳ', 'γὰρ', 'καιρῷ', 'ἐσμεν', ')', 'αὐτούς', 'τε', 'ἄνδρας', 'ἀγαθοὺς', 'εἶναι', 'καὶ', 'τοὺς', 'ἄλλους', 'παρακαλεῖν', '.'] γὰρ
1952 ['ποταμός', 'γάρ']
new ['πάντες', 'γὰρ', 'ποταμοί', ',', 'ἢν', 'καὶ', 'πρόσω', 'τῶν', 'πηγῶν', 'ἄποροι', 'ὦσι', ',', 'προσιοῦσι', 'πρὸς', 'τὰς', 'πηγὰς'

new ['καὶ', 'ἄρδην', 'πάντες', 'πλὴν', 'ὀλίγων', 'ἔξω', 'ἦσαν', ',', 'καὶ', 'Ἐτεόνικος', 'εἱστήκει', 'παρὰ', 'τὰς', 'πύλας', 'ὡς', 'ὁπότε', 'ἔξω', 'γένοιντο', 'πάντες', 'συγκλείσων', 'τὰς', 'πύλας', 'καὶ', 'τὸν', 'μοχλὸν', 'ἐμβαλῶν', '.'] παρὰ
2231 ['στρατιά']
new ['ὁ', 'δὲ', 'Σεύθης', 'ἀκούσας', 'ἥκοντα', 'πάλιν', 'πέμψας', 'πρὸς', 'αὐτὸν', 'κατὰ', 'θάλατταν', 'Μηδοσάδην', 'ἐδεῖτο', 'τὴν', 'στρατιὰν', 'ἄγειν', 'πρὸς', 'ἑαυτόν', ',', 'ὑπισχνούμενος', 'αὐτῷ', 'ὅ', 'τι', 'ᾤετο', 'λέγων', 'πείσειν', '.'] στρατιὰν
2232 ['παρά']
new ['ὁ', 'δ’', 'Ἀρύστας', ',', 'ἐπεὶ', 'παρ’', 'αὐτὸν', 'φέρων', 'τὸ', 'κέρας', 'ὁ', 'οἰνοχόος', 'ἧκεν', ',', 'εἶπεν', 'ἰδὼν', 'τὸν', 'Ξενοφῶντα', 'οὐκέτι', 'δειπνοῦντα', ',', 'ἐκείνῳ', ',', 'ἔφη', ',', 'δός', '·'] παρ’
2233 ['ἐκ', 'ἀπό', 'φιλία', 'ἐντεῦθεν', 'ἡμέρα']
new ['ἐντεῦθεν', 'ὁ', 'Ἡρακλείδης', 'ἠχθέσθη', 'τε', 'καὶ', 'ἔδεισε', 'μὴ', 'ἐκ', 'τῆς', 'Σεύθου', 'φιλίας', 'ἐκβληθείη', ',', 'καὶ', 'ὅ', 'τι', 'ἐδύνατο', 'ἀπὸ', 'ταύτης', 'τῆς', 'ἡμέρας', 'Ξενοφῶντα

new ['τότε', 'δὴ', 'καὶ', 'ἐγνώσθη', 'ὅτι', 'οἱ', 'βάρβαροι', 'τὸν', 'ἄνθρωπον', 'ὑποπέμψειαν', ',', 'ὀκνοῦντες', 'μὴ', 'οἱ', 'Ἕλληνες', 'διελόντες', 'τὴν', 'γέφυραν', 'μείναιεν', 'ἐν', 'τῇ', 'νήσῳ', 'ἐρύματα', 'ἔχοντες', 'ἔνθεν', 'μὲν', 'τὸν', 'Τίγρητα', ',', 'ἔνθεν', 'δὲ', 'τὴν', 'διώρυχα', '·'] ἐν
2471 ['μέν']
new ['ἐμοὶ', 'μὲν', 'ταῦτα', 'πάντα', 'ἐνθυμουμένῳ', 'οὕτω', 'δοκεῖ', 'θαυμαστὸν', 'εἶναι', 'τὸ', 'σὲ', 'ἡμῖν', 'ἀπιστεῖν', 'ὥστε', 'καὶ', 'ἥδιστ’', 'ἂν', 'ἀκούσαιμι', 'τὸ', 'ὄνομα', 'τίς', 'οὕτως', 'ἐστὶ', 'δεινὸς', 'λέγειν', 'ὥστε', 'σε', 'πεῖσαι', 'λέγων', 'ὡς', 'ἡμεῖς', 'σοι', 'ἐπιβουλεύομεν', '.'] μὲν
2472 ['ἐκ', 'μέν']
new ['ἐκ', 'τούτου', 'ᾑρέθησαν', 'ἄρχοντες', 'ἀντὶ', 'μὲν', 'Κλεάρχου', 'Τιμασίων', 'Δαρδανεύς', ',', 'ἀντὶ', 'δὲ', 'Σωκράτους', 'Ξανθικλῆς', 'Ἀχαιός', ',', 'ἀντὶ', 'δὲ', 'Ἀγίου', 'Κλεάνωρ', 'Ἀρκάς', ',', 'ἀντὶ', 'δὲ', 'Μένωνος', 'Φιλήσιος', 'Ἀχαιός', ',', 'ἀντὶ', 'δὲ', 'Προξένου', 'Ξενοφῶν', 'Ἀθηναῖος', '.'] ἐκ
new ['ἐκ', 'τούτου', 'ᾑρέθησαν', 'ἄρχοντες',

2698 ['φίλος']
new ['Ἀριαῖος', 'δέ', ',', 'ὃν', 'ἡμεῖς', 'ἠθέλομεν', 'βασιλέα', 'καθιστάναι', ',', 'καὶ', 'ἐδώκαμεν', 'καὶ', 'ἐλάβομεν', 'πιστὰ', 'μὴ', 'προδώσειν', 'ἀλλήλους', ',', 'καὶ', 'οὗτος', 'οὔτε', 'τοὺς', 'θεοὺς', 'δείσας', 'οὔτε', 'Κῦρον', 'τεθνηκότα', 'αἰδεσθείς', ',', 'τιμώμενος', 'μάλιστα', 'ὑπὸ', 'Κύρου', 'ζῶντος', 'νῦν', 'πρὸς', 'τοὺς', 'ἐκείνου', 'ἐχθίστους', 'ἀποστὰς', 'ἡμᾶς', 'τοὺς', 'Κύρου', 'φίλους', 'κακῶς', 'ποιεῖν', 'πειρᾶται', '.'] φίλους
2699 ['ὁδός', 'μέν']
new ['ὁπότε', 'μὲν', 'οὖν', 'τοὺς', 'πρώτους', 'κωλύοιεν', ',', 'Ξενοφῶν', 'ὄπισθεν', 'ἐκβαίνων', 'πρὸς', 'τὰ', 'ὄρη', 'ἔλυε', 'τὴν', 'ἀπόφραξιν', 'τῆς', 'ὁδοῦ', 'τοῖς', 'πρώτοις', 'ἀνωτέρω', 'πειρώμενος', 'γίγνεσθαι', 'τῶν', 'κωλυόντων', ',', 'ὁπότε', 'δὲ', 'τοῖς', 'ὄπισθεν', 'ἐπιθοῖντο', ',', 'Χειρίσοφος', 'ἐκβαίνων', 'καὶ', 'πειρώμενος', 'ἀνωτέρω', 'γίγνεσθαι', 'τῶν', 'κωλυόντων', 'ἔλυε', 'τὴν', 'ἀπόφραξιν', 'τῆς', 'παρόδου', 'τοῖς', 'ὄπισθεν', '·'] ὁδοῦ
new ['ὁπότε', 'μὲν', 'οὖν', 'τοὺς', 'πρώτους', 'κω

In [13]:
nomlist4

['ἀδελφός',
 'ἔχει',
 'ἔχουσι',
 'ἦν',
 'ἦσαν',
 'παύει',
 'παύουσι',
 'πέμπει',
 'πέμπουσι',
 'ποταμός',
 'στρατηγός',
 'ἄνθρωπος',
 'ἄξιος',
 'ἀπό',
 'δίκαιος',
 'Ἑλλήσποντος',
 'ἐκ',
 'μακρός',
 'μικρός',
 'πόλεμος',
 'πολέμιος',
 'φίλος',
 'ἀγαθός',
 'ἄγγελος',
 'ἵππος',
 'καλός',
 'κίνδυνος',
 'λίθος',
 'ἐν',
 'δένδρον',
 'παρά',
 'δῆλος',
 'δῶρον',
 'εἰς',
 'πεδίον',
 'στάδιον']

In [8]:

from tabulate import tabulate
from prettytable import PrettyTable


            
outf = open('parads/test.md','w')

nparad1 = ['s---Zv','s---Zn','s---Zg','s---Zd','s---Za',
           'p---Zv','p---Zn','p---Zg','p---Zd','p---Za']
nparads = ['s---mv','s---mn','s---mg','s---md','s---ma',
            'p---mv','p---mn','p---mg','p---md','p---ma',
            's---fv','s---fn','s---fg','s---fd','s---fa',
            'p---fv','p---fn','p---fg','p---fd','p---fa',
            's---nv','s---nn','s---ng','s---nd','s---na',
            'p---nv','p---nn','p---ng','p---nd','p---na']

def printnounrow(useparad,curparad,curparadf,index,outf):
    if(re.search(',',curparad[useparad[index]])):
        rowvalue1 = curparad[useparad[index]]
    else:
        rowvalue1 = re.sub('\s+[0-9]+$','',curparad[useparad[index]])
    if(re.search(',',curparad[useparad[index+5]])):
        rowvalue2 = curparad[useparad[index+5]]
    else:
        rowvalue2 = re.sub('\s+[0-9]+$','',curparad[useparad[index+5]])
    print('| ' + defart_forms[useparad[index]],
          rowvalue1,
          #curparad[useparad[index]],
          curparadf[useparad[index]],
          defart_forms[useparad[index+5]],
          rowvalue2,
          #curparad[useparad[index+5]],
          str(curparadf[useparad[index+5]])+' |',file=outf,sep="|")

def printadjrow(useparad,curparad,curparadf,index,outf):
    curcasenum = useparad[index][5].upper()
    if(re.search(',',curparad[useparad[index]])):
        rowvalue1 = curparad[useparad[index]]
    else:
        rowvalue1 = re.sub('\s+[0-9]+$','',curparad[useparad[index]])
    if(re.search(',',curparad[useparad[index+10]])):
        rowvalue2 = curparad[useparad[index+10]]
    else:
        rowvalue2 = re.sub('\s+[0-9]+$','',curparad[useparad[index+10]])
    if(re.search(',',curparad[useparad[index+20]])):
        rowvalue3 = curparad[useparad[index+20]]
    else:
        rowvalue3 = re.sub('\s+[0-9]+$','',curparad[useparad[index+20]])
    print('| ' + curcasenum, 
          #curparad[useparad[index]],
          rowvalue1,
          curparadf[useparad[index]],
          #curparad[useparad[index+10]],
          rowvalue2,
          curparadf[useparad[index+10]],
          #curparad[useparad[index+20]],
          rowvalue3,
          str(curparadf[useparad[index+20]])+' |',file=outf,sep="|")

genders = ['m','f','n']
def printadjparad(curmasc,outf):
    if(not curmasc in lemma_pos_forms):
        print(curmasc,0)
        return

    
    curforms = lemma_pos_forms[curmasc]
    curparad = {}
    curparadf = {}
    for foo in nparads:
        for boo in curforms:
            m = re.search('('+foo+')',boo)
            if(m):
                #print(boo,lemma_pos_forms[curmasc],lemma_pos_forms[curmasc][boo])
                if(foo in curparadf):
                    curparadf[foo] = curforms[boo] + curparadf[foo]
                    if(curparad[foo] == 'NA'):
                        curparad[foo] = re.sub('[^:]+:','',boo)   + ' ' + str(curforms[boo])
                    else:   
                        curparad[foo] = curparad[foo] + ', ' + re.sub('[^:]+:','',boo)  + ' ' + str(curforms[boo])
                        print('hasnum',foo,curparad[foo],curparadf[foo])
                else:
                    curparadf[foo] = curforms[boo]
                    curparad[foo] = re.sub('[^:]+:','',boo) + ' ' + str(curforms[boo])
                print(foo,boo,curparadf[foo])
        for foo in nparads:
            if(not foo in curparad):
                curparad[foo] = 'NA'
                curparadf[foo] = 0
                #print('usetag',usetag)
                
    for foo in curparad:
        print('adj',foo,curparad[foo],curparadf[foo])
    
        
    print('\n',file=outf)
    print('##',curmasc,greeklems[curmasc],file=outf)
    
    print('|case | masc | # | fem  | #| neut  | #|',file=outf)
    print('| --- | --- | --- | --- | --- | --- | ---  |',file=outf)
    print('| Sing | |  |  | |  |   |',file=outf)
    
    for i in range(0,10):
        if(i == 5):
            print('| Plural | |  |  | |  |   |',file=outf)
        print(nparads[i],nparads[i+9],nparads[i+18])
        
        printadjrow(nparads,curparad,curparadf,i,outf)



def printnomparad(curgend,curmasc,outf):
    useparad = []
    for foo in nparad1:
        useparad.append(re.sub('Z',curgend,foo))
    
    if(not curmasc in lemma_pos_forms):
        print(curmasc,0)
        return

    curparad = {}
    curparadf = {}
    for foo in useparad:
        for boo in lemma_pos_forms[curmasc]:
            if(re.search(foo,boo)):
                #print(boo,lemma_pos_forms[curmasc],lemma_pos_forms[curmasc][boo])
                curparad[foo] = re.sub('[^:]+:','',boo) + ' ' + str(lemma_pos_forms[curmasc][boo])
                print('foo',foo,curparad[foo])
                curparadf[foo] = lemma_pos_forms[curmasc][boo]
                #print(foo,boo,curparadf[foo])
        for foo in useparad:
            if(not foo in curparad):
                curparad[foo] = 'NA'
                curparadf[foo] = 0
                
        
    
    
    print('| sg | | # | pl | | #|',file=outf)
    print('| --- | --- | --- | --- | --- | --- |',file=outf)
    
    
    printnounrow(useparad,curparad,curparadf,0,outf)
    printnounrow(useparad,curparad,curparadf,1,outf)
    printnounrow(useparad,curparad,curparadf,2,outf)
    printnounrow(useparad,curparad,curparadf,3,outf)
    printnounrow(useparad,curparad,curparadf,4,outf)

    print('\n\n',file=outf)

def nomgenders(curnom,outf):
    gendtable = {}
    print('\n',file=outf)
    print('##',curnom,greeklems[curnom],file=outf)

    for boo in lemma_pos_forms[curnom]:
        if(boo[0] in ['n','p']):
            curgend = boo[6]
            if(curgend in gendtable):
                gendtable[curgend] = gendtable[curgend] + lemma_pos_forms[curnom][boo]
            else:
                gendtable[curgend] = lemma_pos_forms[curnom][boo]
                
    for foo in gendtable:
        printnomparad(foo,curnom,outf)

            

    
curmasc = 'πολέμιος'
printadjparad(curmasc,outf)
    
curmasc = 'πᾶς'
printadjparad(curmasc,outf)
    
curmasc = 'φίλος'
printadjparad(curmasc,outf)

curmasc = 'στάδιον'
nomgenders(curmasc,outf)
curmasc = 'δῶρον'
nomgenders(curmasc,outf)
curmasc = 'γυνή'
nomgenders(curmasc,outf)
curmasc = 'ἀδελφός'
nomgenders(curmasc,outf)
curmasc = 'ποταμός'
nomgenders(curmasc,outf)

curmasc = 'κώμη'
nomgenders(curmasc,outf)

curmasc = 'μάχη'
nomgenders(curmasc,outf)

curmasc = 'σκηνή'
nomgenders(curmasc,outf)

curmasc = 'σπονδή'
nomgenders(curmasc,outf)

curmasc = 'φυγή'
nomgenders(curmasc,outf)

curmasc = 'φυλακή'
nomgenders(curmasc,outf)

curmasc = 'ἀνήρ'
nomgenders(curmasc,outf)

curmasc = 'παῖς'
nomgenders(curmasc,outf)

printadjparad('ἐπιτήδειος',outf)
nomgenders('στάδιον',outf)
outf.close()


s---mn a-s---mn-:πολέμιος 4
s---mg a-s---mg-:πολεμίου 1
hasnum s---mg πολεμίου 1, πολεμιωτάτου 1 2
s---mg a-s---mgs:πολεμιωτάτου 2
s---ma n-s---ma-:πολέμιον 1
hasnum s---ma πολέμιον 1, πολέμιον 1 2
s---ma a-s---ma-:πολέμιον 2
p---mn a-p---mnc:πολεμιώτεροι 1
hasnum p---mn πολεμιώτεροι 1, πολέμιοι 79 80
p---mn a-p---mn-:πολέμιοι 80
p---mg a-p---mg-:πολεμίων 45
p---md a-p---md-:πολεμίοις 26
p---ma a-p---ma-:πολεμίους 54
hasnum p---ma πολεμίους 54, πολεμίους 1 55
p---ma n-p---ma-:πολεμίους 55
s---fn a-s---fn-:πολεμία 1
s---fg a-s---fg-:πολεμίας 4
s---fd a-s---fd-:πολεμίᾳ 5
s---fa a-s---fa-:πολεμίαν 2
p---fn a-p---fn-:πολέμιαι 1
p---fg a-p---fg-:πολεμίων 2
s---nn a-s---nn-:πολέμιον 1
s---ng a-s---ng-:πολεμίου 1
s---na a-s---na-:πολέμιον 2
p---ng a-p---ng-:πολεμίων 1
p---na a-p---na-:πολέμια 1
adj s---mv NA 0
adj s---mn πολέμιος 4 4
adj s---mg πολεμίου 1, πολεμιωτάτου 1 2
adj s---md NA 0
adj s---ma πολέμιον 1, πολέμιον 1 2
adj p---mv NA 0
adj p---mn πολεμιώτεροι 1, πολέμιοι 79 80
adj p---mg 

In [5]:
'ὠφέλιμος' in oldlemmas

anttb = '/Users/gcrane/github/Daphne/data/annotation/latest/tlg0011/tlg002/tlg0011.tlg002.daphne_tb-grc1.conllu'
antvoc = {}
f = open(anttb)

curln = 0
for l in f:
    m = re.search('Ref=([0-9]+)',l)
    if(m):
        curln = int(m[1])
    if(not re.search('^[0-9]+\s',l)):
        continue
    if(curln < 163 or curln > 581):
        continue
    if(re.search('PUNCT',l)):
        continue
    args = l.split()

    lemma = args[2]
    if(lemma in antvoc):
        antvoc[lemma] = 1 + antvoc[lemma]
    else:
        antvoc[lemma] = 1
    #print(curln,lemma)

for foo in sorted(antvoc,key=antvoc.get,reverse=True):
    if(not foo in oldlemmas):
        if(foo in csvocab):
            curless = csvocab[foo]
        else:
            curless = 'notCS'
        print(foo,antvoc[foo],'miss',curless)
    #else:
    #    print(foo,antvoc[foo],'hit')
f.close()

NameError: name 'oldlemmas' is not defined

In [6]:
i = 0
for foo in csvocab:
    if(not foo in tbvoc):
        i = i + 1
        print(i,foo,csvocab[foo])

NameError: name 'tbvoc' is not defined

In [36]:
i = 0
j = 0
seentokens = 0
for foo in sorted(tbvoc,key=tbvoc.get,reverse=True):
    if(foo[0].isupper()):
        continue
    i= i + 1
    if(i>5000):
        break
    if(not foo in csvocab):
        cv = 'NA'
    else:
        cv = csvocab[foo]
        
    if(   cv == 'NA'):
        j = j + 1
        seentokens = seentokens + tbvoc[foo]
        if(j<500):
            print(i,j,seentokens,tbvoc[foo],foo,cv)
print(tottokens,seentokens,seentokens/tottokens,.8*tottokens)

1 1 6522 6522 ὁ NA
65 2 6644 122 ἀπέρχομαι NA
69 3 6755 111 δεῖ NA
76 4 6860 105 ὥστε NA
107 5 6938 78 οἴομαι NA
136 6 6995 57 ἀνίστημι NA
141 7 7050 55 σταθμός NA
149 8 7101 51 τρεῖς NA
152 9 7151 50 ἀμφί NA
161 10 7197 46 δύο NA
179 11 7237 40 εἴκοσι NA
187 12 7275 38 προσέρχομαι NA
189 13 7313 38 καταλαμβάνω NA
192 14 7350 37 οἴχομαι NA
195 15 7386 36 καταλείπω NA
198 16 7422 36 τοσοῦτος NA
205 17 7457 35 ἐξέρχομαι NA
206 18 7492 35 ἀπαγγέλλω NA
208 19 7526 34 παραγγέλλω NA
213 20 7560 34 παρέρχομαι NA
218 21 7593 33 ἅπας NA
223 22 7624 31 ἄκρον NA
227 23 7655 31 παραδίδωμι NA
231 24 7685 30 καταβαίνω NA
232 25 7715 30 ὑποζύγιον NA
240 26 7744 29 ἀναβαίνω NA
241 27 7773 29 συλλέγω NA
250 28 7801 28 τριάκοντα NA
259 29 7828 27 ἐπιτίθημι NA
261 30 7855 27 μηδείς NA
263 31 7882 27 συνέρχομαι NA
264 32 7908 26 πρέσβυς NA
265 33 7934 26 καθίστημι NA
268 34 7960 26 ὅσπερ NA
275 35 7985 25 τέσσαρες NA
283 36 8009 24 ἐπέρχομαι NA
313 37 8031 22 διαπράσσω NA
314 38 8053 22 χιών NA
319 39 807

787 319 10979 7 ὀρχέομαι NA
788 320 10986 7 μισθοφορά NA
789 321 10993 7 σωφρονέω NA
790 322 11000 7 αἰτία NA
791 323 11007 7 δωρέω NA
793 324 11013 6 διατίθημι NA
794 325 11019 6 ἐκβάλλω NA
795 326 11025 6 συμπράσσω NA
798 327 11031 6 ἀκρόπολις NA
799 328 11037 6 παράδεισος NA
800 329 11043 6 θηρίον NA
801 330 11049 6 ἐρυμνός NA
802 331 11055 6 ἐξέτασις NA
803 332 11061 6 χρύσεος NA
804 333 11067 6 θεωρέω NA
805 334 11073 6 προβάλλω NA
806 335 11079 6 ἐκλείπω NA
807 336 11085 6 κατακόπτω NA
808 337 11091 6 ὑπολείπω NA
809 338 11097 6 κατατίθημι NA
810 339 11103 6 ἀλέξω NA
811 340 11109 6 λυπέω NA
812 341 11115 6 καθεύδω NA
813 342 11121 6 στρατηγέω NA
814 343 11127 6 ὑποψία NA
815 344 11133 6 ναύαρχος NA
816 345 11139 6 τετρακόσιοι NA
817 346 11145 6 ἀναστρέφω NA
820 347 11151 6 ἐπιλείπω NA
821 348 11157 6 χιλός NA
824 349 11163 6 προσέχω NA
825 350 11169 6 φοῖνιξ NA
827 351 11175 6 χρήσιμος NA
828 352 11181 6 τρισχίλιοι NA
829 353 11187 6 ἀμαχεί NA
830 354 11193 6 ἐξοπλίζω NA
831 355

In [14]:
csvocab

{'ἀδελφός': ['1'],
 'ἔχει': ['1'],
 'ἔχουσι': ['1'],
 'ἦν': ['1'],
 'ἦσαν': ['1'],
 'παύει': ['1'],
 'παύουσι': ['1'],
 'πέμπει': ['1'],
 'πέμπουσι': ['1'],
 'ποταμός': ['1'],
 'στρατηγός': ['1'],
 'ἄνθρωπος': ['2'],
 'ἄξιος': ['2'],
 'ἀπό': ['2'],
 'δίκαιος': ['2'],
 'Ἑλλήσποντος': ['2'],
 'ἐκ': ['2'],
 'μακρός': ['2'],
 'μικρός': ['2', '42', '42'],
 'πόλεμος': ['2'],
 'πολέμιος': ['2'],
 'φίλος': ['2'],
 'ἀγαθός': ['3', '42', '42', '42', '42'],
 'ἄγγελος': ['3'],
 'ἄγω': ['3'],
 'γράφω': ['3'],
 'ἐθέλω': ['3'],
 'ἵππος': ['3'],
 'καί': ['3'],
 'καλός': ['3', '28', '42', '42'],
 'κίνδυνος': ['3'],
 'λίθος': ['3'],
 'λύω': ['3'],
 'παύω': ['3'],
 'δέ': ['4'],
 'ἐν': ['4'],
 'δένδρον': ['4'],
 'παρά': ['4'],
 'δῆλος': ['4'],
 'δῶρον': ['4'],
 'εἰς': ['4'],
 'πεδίον': ['4'],
 'στάδιον': ['4'],
 'γάρ': ['6'],
 'σπονδή': ['6'],
 'κώμη': ['6'],
 'μάχη': ['6'],
 'φεύγω': ['6'],
 'οὐ': ['6'],
 'φυγή': ['6'],
 'φυλακή': ['6'],
 'σκηνή': ['6'],
 'φυλάσσω': ['6'],
 'ἀγορά': ['7'],
 'ἀρχή': ['7']

In [15]:
paradpats = {}
paradpats['ostem_m1'] = 'form="([^"]+)" lemma="([^"]+ός)" postag="(n.....m[^"]+)'
paradpats['h_hs_h'] = 'form="([^"]+)" lemma="([^"]+[^ίέριε][ηή])" postag="(n.....f[^"]+)'
paradpats['h_hs_La'] = 'form="([^"]+)" lemma="([^"]+[ίέρ][ηα])" postag="(n.....f[^"]+)'
paradpats['defart_m'] = 'form="([^"]+)" lemma="(ὁ)" postag="(l.....m[^"]+)'
paradpats['defart_n'] = 'form="([^"]+)" lemma="(ὁ)" postag="(l.....n[^"]+)'
paradpats['defart_f'] = 'form="([^"]+)" lemma="(ὁ)" postag="(l.....f[^"]+)'
paradpats['wstem_3.pia'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v3.pia[^"]+)'
paradpats['wstem_pia'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v..pia[^"]+)'
paradpats['wstem_iia'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v..iia[^"]+)'
paradpats['wstem_pna'] = 'form="([^"]+)" lemma="([^"]+[^άέό]ω)" postag="(v..pia[^"]+)'
paradpats['regfut_fia'] = 'form="([^"]+)" lemma="([^"ῖῦῶ]+ω)" postag="(v..fia[^"]+)'
paradpats['regfut_fna'] = 'form="([^"]+)" lemma="([^"ῖῦῶ]+ω)" postag="(v..fna[^"]+)'
#lesson 10 covers 1st and 2nd aorists both
paradpats['aor_aia'] = 'form="([^"]+[^ηω])" lemma="([^"]+ω)" postag="(v..aia[^"]+)'
paradpats['aor_ana'] = 'form="([^"]+[^ηω])" lemma="([^"]+ω)" postag="(v..ana[^"]+)'

paradpats['ostem_m2'] = 'form="([^"]+)" lemma="([^"]+ος)" postag="(n.....m[^"]+)'
paradpats['ostem_n'] = 'form="([^"]+)" lemma="([^"]+[οό]ν)" postag="(n.....n[^"]+)'

paradpats['adj1_m'] = 'form="([^"]+)" lemma="([^"]+[οό]ς)" postag="(a.....m[^"]+)'
paradpats['adj1_n'] = 'form="([^"]+)" lemma="([^"]+[οό]ς)" postag="(a.....n[^"]+)'
paradpats['adj1_f'] = 'form="([^"]+)" lemma="([^"]+[οό]ς)" postag="(a.....f[^"]+)'

paradpats['hos_m'] = 'form="([^"]+)" lemma="(ὅς)" postag="(p.....m[^"]+)'
paradpats['hos_n'] = 'form="([^"]+)" lemma="(ὅς)" postag="(p.....n[^"]+)'
paradpats['seautou_m'] = 'form="([^"]+)" lemma="(σεαυτοῦ)" postag="(p.....m[^"]+)'
paradpats['emautou_m'] = 'form="([^"]+)" lemma="(ἐμαυτοῦ)" postag="(p.....m[^"]+)'
paradpats['autos_m'] = 'form="([^"]+)" lemma="(αὐτός)" postag="(a.....m[^"]+)'
paradpats['hode_m'] = 'form="([^"]+)" lemma="(ὅδε)" postag="(a.....m[^"]+)'

#recessive
paradpats['w_p_mp'] = 'form="([^"]+)" lemma="([^"]+ω)" postag="(v..[fp]p[mep][^"]+)'
paradpats['w_a_m'] = 'form="([^"]+)" lemma="([^"]+ω)" postag="(v..ap[me][^"]+)'

lemma_class = {}

lesson01 = ['ostem_m1','defart_m','wstem_3.pia','hos_m','autos_m','hode_m','seautou_m','emautou_m','w_p_mp','w_a_m']
lesson02 = ['ostem_m2','adj1_m','w_p_mp','w_a_m']
lesson03 = ['wstem_pia','wstem_pna']
lesson04 = ['ostem_n','adj1_n','defart_n','hos_n']
lesson06 = ['h_hs_h','adj1_f','defart_f']
lesson07 = ['h_hs_La']
lesson08 = ['wstem_iia']
lesson09 = ['regfut_fia','regfut_fna']
lesson10 = ['aor_aia','aor_ana']

curlesson = lesson06

def add_lemma(pclass,lemma):
    if(not pclass in lemma_class):
        lemma_class[pclass] = {}
    curdict = lemma_class[pclass]
    if(lemma in curdict):
        curdict[lemma] = curdict[lemma] + 1
    else:
        curdict[lemma] = 1
    #lemma_class[pclass] = curdict

def addparad(curpclass,l):
    m = re.search(paradpats[curpclass],l)
    if(m):
        curform = m[1]
        curlemma = m[2]
        curpostag = m[3]
        add_lemma(curpclass,curlemma)
    
def getparadpats(fname):
    global paradpats
    
    f = open(fname)
    for l in f:
        l = re.sub('\n','',l)
        
        for foo in curlesson:
            #print('doing',foo)
            addparad(foo,l)
    
    
    f.close()
 
getparadpats(curtb)

k = 0
for foo in curlesson:
    curdict = lemma_class[foo]

    print(foo,len(curdict))

    i = 0
    j = 0
    for foo in sorted(curdict,key=curdict.get,reverse=True):
        i = i + 1
        j = j + curdict[foo]
        k = k + curdict[foo]
        if(not foo in lexicalentries):
            shortdef = 'nodef'
        else:
            shortdef = lexicalentries[foo]
        print(i,j,k,foo,curdict[foo],shortdef)


h_hs_h 138
1 87 87 κώμη 87 country town
2 121 121 σπονδή 34 drink-offering; (pl.) truce
3 154 154 μάχη 33 battle
4 185 185 ἀνάγκη 31 force
5 214 214 γυνή 29 a woman
6 238 238 δίκη 24 (custom
7 260 260 ἀρχή 22 a beginning
8 281 281 πύλη 21 one wing of a pair of double gates
9 300 300 φυλακή 19 a watching
10 318 318 γνώμη 18 a means of knowing
11 332 332 σκηνή 14 a covered place
12 346 346 κεφαλή 14 the head
13 359 359 οἰκουμένη 13 the inhabited world
14 372 372 κραυγή 13 a crying
15 383 383 λόγχη 11 a spear-head
16 393 393 πηγή 10 running waters
17 402 402 ἀρετή 9 goodness
18 411 411 τιμή 9 that which is paid in token of worth
19 420 420 πέλτη 9 a small light shield
20 428 428 ὑπερβολή 8 a throwing beyond
21 436 436 δείλη 8 afternoon
22 444 444 σφενδόνη 8 a sling
23 451 451 κριθή 7 barley-corns
24 458 458 φυγή 7 flight
25 465 465 Κάλπη 7 nodef
26 471 471 ἐπιβουλή 6 a plan against
27 477 477 Θρᾴκη 6 Thrace
28 482 482 μελίνη 5 millet
29 487 487 νίκη 5 victory
30 492 492 ἐπιστολή 5 a messa

In [28]:
import icu

icu_transliterator = icu.Transliterator.createInstance("Any-Latin")
value = "μῆνιν"
translit_value = icu_transliterator.transliterate(value)
print(translit_value)


mē̂nin


In [29]:
i = 0
for foo in sorted(greeksentlens,key=greeksentlens.get):
    i = i + 1
    print(i,foo,len(greeksentlems[foo]),greeksentlems[foo])
    if(i>10):
        break

1 3668 2 ['φημί', '·']
2 158 3 ['οὗτος', 'λέγω', '·']
3 237 3 ['οὔτε', 'ἀποφεύγω', '·']
4 366 3 ['φημί', 'Ὀρόντας', '.']
5 681 3 ['ἀποκρίνω', 'Κλέαρχος', '·']
6 835 3 ['τίς', 'μένω', ';']
7 972 3 ['καί', 'λέγω', '·']
8 1099 3 ['τίς', 'κατάκειμαι', ';']
9 1264 3 ['ἀνατείνω', 'πᾶς', '.']
10 1280 3 ['δοκέω', 'οὗτος', '.']
11 1423 3 ['ἔξεστι', 'ὁράω', '·']


In [31]:
genabsolutes

['    <word id="9004803" form="Κύρου" lemma="Κῦρος" postag="n-s---mg-" head="9004802" relation="SBJ"/>',
 '    <word id="9005645" form="αὐτοῦ" lemma="αὐτός" postag="a-s---mg-" head="9005643" relation="SBJ"/>']

In [47]:
getfeats('9004803')

9004803 


('', '')

In [41]:
formdict['9004803']

'    <word id="9004803" form="Κύρου" lemma="Κῦρος" postag="n-s---mg-" head="9004802" relation="SBJ"/>'

In [22]:
sorted(preplist)

['διά',
 'εἰς',
 'κατά',
 'μά',
 'μέχρι',
 'μετά',
 'μεταξύ',
 'νή',
 'πέραν',
 'παρά',
 'περί',
 'πλήν',
 'πλησίον',
 'πρό',
 'πρός',
 'σύν',
 'χωρίς',
 'ἀμφί',
 'ἀνά',
 'ἀντί',
 'ἀντίον',
 'ἀπό',
 'ἄνευ',
 'ἄχρι',
 'ἐκ',
 'ἐν',
 'ἐντός',
 'ἐπέκεινα',
 'ἐπί',
 'ἔμπροσθεν',
 'ἕνεκα',
 'ἕως',
 'ὑπέρ',
 'ὑπό']

In [23]:
print(len(aobjverbs))
for foo in sorted(aobjverbs,key=aobjverbs.get,reverse=True):
    print(aobjverbs[foo],foo)

496


TypeError: '<' not supported between instances of 'dict' and 'dict'

In [22]:
i = 0
outf = open('wstem-pia.txt','w')
for form in formdict:
    if(not re.search('^1\.',citdict[form])):
        continue
    if( not re.search('lemma="[^"]+ω"',formdict[form])):
        continue
    
    if( not re.search('pia',formdict[form])):
        continue
    
    if(re.search('(άω|έω|όω)"',formdict[form])):
        continue
        
    
    curpostag,curlemma,curform,curhead,currel = getfeats(form)  
    if(not curlemma in lexicalentries):
        continue
    i = i + 1
    print(i,curpostag,citdict[form],lexicalentries[curlemma],curlemma,sep='\t',file=outf)
outf.close()

In [11]:
i = 0
tenses = {}
def addentry(curitem,curdict):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
    else:
        curdict[curitem] = 1
        
outf = open('presimper.txt','w')
for foo in formdict:
    curcit = citdict[foo]
    if(not re.search('^1\.',curcit)):
        continue
    m = re.search('v..([fapi]ia)',formdict[foo])
    if(not m):
        continue
        
    curpostag,curlemma,curform,curhead,currel = getfeats(foo)  

    if(re.search('(έω|άω|όω)$',curlemma)):
        continue
    if(not re.search('(ω)$',curlemma)):
        continue
    addentry(m[1],tenses)
        
    if(curlemma in lexicalentries):
        curgloss = lexicalentries[curlemma]
    else:
        curgloss = 'nodef'
    i = i + 1
    print(i,curform,curcit,curlemma,curgloss,sep='\t',file=outf)
    
outf.close()

In [12]:
tenses

{'iia': 115, 'pia': 107, 'aia': 131, 'fia': 11}

In [12]:
len(tenses)

3

In [7]:
conditionals

{}

In [15]:
print(len(datmeansexamp))
datmeansexamp

342


[['αὐτῷ', 'ADV', 'ἦν', '8996316', '1.1.7'],
 ['ᾗ', 'ADV', 'ἐδύνατο', '8996847', '1.2.4'],
 ['πλοίοις', 'ADV', 'ἐζευγμένη', '8996903', '1.2.5'],
 ['Κύρῳ', 'ADV', 'ἦν', '8996966', '1.2.7'],
 ['μάχῃ', 'ADV', 'ἡττηθεὶς', '8997100', '1.2.9'],
 ['οἴνῳ', 'ADV', 'κεράσας', '8997387', '1.2.13'],
 ['ὑστεραίᾳ', 'ADV', 'ἧκεν', '8997816', '1.2.21'],
 ['τριήρεσι', 'ADV', 'καταδύσῃ', '8999108', '1.3.17'],
 ['τοῖς', 'ADV', 'ἦν', '8999325', '1.3.21'],
 ['αἷς', 'ADV', 'ἐπολιόρκει', '8999473', '1.4.2'],
 ['βίᾳ', 'ADV', 'παρελθεῖν', '8999609', '1.4.4'],
 ['τριήρεσι', 'ADV', 'διώκει', '8999804', '1.4.7'],
 ['πεζῇ', 'ADV', 'γένοιτο', '9000487', '1.4.18'],
 ['ποσὶ', 'ADV', 'ἀπέσπα', '9000718', '1.5.3'],
 ['δρόμῳ', 'ADV', 'αἴρουσα', '9000719', '1.5.3'],
 ['πτέρυξιν', 'ADV', 'αἴρουσα', '9000723', '1.5.3'],
 ['κύκλῳ', 'ADV', 'περιερρεῖτο', '9000790', '1.5.4'],
 ['σχεδίαις', 'ADV', 'διαβαίνοντες', '9001180', '1.5.10'],
 ['ἡμέρᾳ', 'ADV', 'ἀφιππεύει', '9001281', '1.5.12'],
 ['ἀξίνῃ', 'ADV', 'ἵησι', '9001332', '1.5